In [1]:
%run "..\general_functions\generalFunctions.ipynb"
%run "..\Landscape slide duplicate\Landscape Replacement Function New.ipynb"

In [2]:
start_time = time.time()

## Data Parameters

In [3]:
client_manuf = ["Danone Group"]
client_brands = ["Actimel","Activia","Danacol","Danone","Danone Griego","Danone Original","Danonino","Oikos","Vitalinea","Yopro","Alpro"]
 
decimals = 2
sign = "After"
currency = '€'
currency = ' '+ currency if sign.lower() == 'after' else  currency + ' '
 
 
categories = ["Yogurt"]
sectors = ["Functionals","Every Day Nutrition","Plant Based","Protein"]
segments = ["Bifidus","Cholesterol","Essential","Greek","Immunity","Kefir","Kids","Light","Plant Based","Protein", "Other Segments"]
subsegments= ["Essential Drink","Essential Spoon","Greek Spoon","Kids Drink","Kids Spoon","Light Drink","Light Spoon", "Other Segments"]
subcategories= []

national = True
customareas= ""
areas = ["NATIONAL", "RETAILER","CHANNEL"]
 
regions_RET  = ["Ahorramas", "Carrefour"]
channels_RET = []
market_RET = []
 
regions_CHAN = ["Discounters", "Hyper", "Super"]
channels_CHAN = []
market_CHAN = []
 
regions_CUST = []
channels_CUST = []
market_CUST = []
 

data_source = "DATA SOURCE: Trade Panel/Retailer Data | Ending December 2024"
 
years = {2022,2023,2024}


ManufOrTopC ="Top Companies"
BrandOrTopB = "Top Brands"
end_date = "2025-01-01"


# subcatg_parent = "Segment"
# subcatg_parent_list = segments
percent = 100000
percentstr="'00 000"

Sections_WO_CB=False

In [4]:
retMarketLength = 3-((not regions_RET)+(not channels_RET)+(not market_RET))
chanMarketLength = 3-((not regions_CHAN)+(not channels_CHAN)+(not market_CHAN))
custMarketLength = 3-((not regions_CUST)+(not channels_CUST)+(not market_CUST))

In [5]:
retailers = regions_RET + channels_RET + market_RET
channels = regions_CHAN + channels_CHAN + market_CHAN
cust = regions_CUST + channels_CUST + market_CUST

### Cleaning Data_frames

In [6]:
loaded_data = {}

# datasets_path = os.getcwd()+"/Landscape Datasets/"
datasets_path = os.getcwd()+"/Landscape Datasets NewEX/"

print(datasets_path)

datasets = os.listdir(datasets_path)
for d in datasets:
    with open(datasets_path+d, 'rb') as handle:
        globals()[d.split('.')[0]] = pd.read_pickle(handle)

c:\Users\BW4SA\Documents\GitHub\Slide-Automate\Landscape slide duplicate/Landscape Datasets NewEX/


In [7]:
# def process_date(manuf_dfs, dfs_total, datacolumn):
#     output = {} 
    
#     for k, df in manuf_dfs.items():
#         df = df.copy()
#         first_col = df.columns[0]
#         df_pivot = df.pivot_table(index=first_col, columns=datacolumn, aggfunc='sum')  
#         df_pivot.columns = [f"{metric} {year}" for metric, year in df_pivot.columns]  # Preserve space
#         df_pivot.reset_index(inplace=True)
        
#         for col in df_pivot.columns:
#             if 'Sales' in col and not pd.api.types.is_numeric_dtype(df_pivot[col]):
#                 df_pivot[col] = pd.to_numeric(df_pivot[col], errors='coerce')
        
#         numeric_cols = df_pivot.select_dtypes(include=['number']).columns
#         years = sorted(df[datacolumn].dropna().unique(), key=list(df[datacolumn]).index)
#         metrics = sorted(set(col.rsplit(" ", 1)[0] for col in numeric_cols), 
#                          key=lambda x: list(df.columns).index(x) if x in df.columns else float('inf'))
#         sorted_columns = [first_col] + [f"{metric} {year}" for year in years for metric in metrics]
#         df_pivot = df_pivot[[col for col in sorted_columns if col in df_pivot.columns]]
#         df_pivot = df_pivot.loc[~(df_pivot.select_dtypes(include=['number']) == 0).all(axis=1)]
#         numeric_cols = df_pivot.select_dtypes(include=['number']).columns  

#         # Merge df_pivot with df_total
#         if k in dfs_total:
#             df_total = dfs_total[k]
#             # Ensure the columns match between df_total and df_pivot
#             common_columns = [col for col in df_pivot.columns if col in df_total.columns]
#             # Perform merge operation on the common columns
            
#             df_pivot = pd.merge(df_pivot, df_total[common_columns], how='left', on=common_columns)
        
#         # Store the result in the output dictionary
#         output[k] = df_pivot
    
#     return output


In [8]:
def process_date(manuf_dfs, dfs_total, datacolumn):
    output = {} 
    
    for k, df in manuf_dfs.items():
        df = df.copy()
        first_col = df.columns[0]
        df_pivot = df.pivot_table(index=first_col, columns=datacolumn, aggfunc='sum')  
        df_pivot.columns = [f"{metric} {year}" for metric, year in df_pivot.columns]  # Preserve space
        df_pivot.reset_index(inplace=True)
        
        for col in df_pivot.columns:
            if 'Sales' in col and not pd.api.types.is_numeric_dtype(df_pivot[col]):
                df_pivot[col] = pd.to_numeric(df_pivot[col], errors='coerce')
        
        numeric_cols = df_pivot.select_dtypes(include=['number']).columns
        years = sorted(df[datacolumn].dropna().unique(), key=list(df[datacolumn]).index)
        metrics = sorted(set(col.rsplit(" ", 1)[0] for col in numeric_cols), 
                         key=lambda x: list(df.columns).index(x) if x in df.columns else float('inf'))
        sorted_columns = [first_col] + [f"{metric} {year}" for year in years for metric in metrics]
        df_pivot = df_pivot[[col for col in sorted_columns if col in df_pivot.columns]]
        df_pivot = df_pivot.loc[~(df_pivot.select_dtypes(include=['number']) == 0).all(axis=1)]
        numeric_cols = df_pivot.select_dtypes(include=['number']).columns  

        # Merge df_pivot with df_total
        if k in dfs_total:
            df_total = dfs_total[k]
            # Ensure the columns match between df_total and df_pivot
            merge_key = df_pivot.columns[0]  # or explicitly set, e.g., 'Segment'
            df_pivot = pd.merge(df_pivot, df_total, how='left', on=merge_key)

        # Store the result in the output dictionary
        output[k] = df_pivot
    
    return output


In [9]:
def secsegclean(inputdic):
    outputdic = {}
    totaloutputdic = {}

    for s in inputdic.keys():
        t = inputdic[s].copy()

        # if not t.empty:
        #     numeric_cols = t.select_dtypes(include='number').columns
        #     total_values = {}

        #     for col in numeric_cols:
        #         if "price" in col.lower():  
        #             total_values[col] = t[col].mean()  
        #         else:
        #             total_values[col] = t[col].sum()  

        #     # Ensure all columns are included in the final row
        #     grand_total_row = pd.DataFrame([["Grand Total"] + 
        #                                     [total_values.get(col, " ") if col in numeric_cols else " " for col in t.columns[1:]]], 
        #                                     columns=t.columns)
        #     t = pd.concat([t, grand_total_row], ignore_index=True)

        # Filter and sort by 'Value Share' columns if they exist
        mod = t[~t[t.columns[0]].astype(str).str.contains('Grand Total', na=False)]
        
        mod = mod.sort_values([col for col in mod.columns if 'Total Value Share' in col], ascending=False)


        tot = t[t[t.columns[0]].astype(str).str.contains('Grand Total', na=False)]

        if not mod.empty:
            outputdic[s] = mod
        if not tot.empty:
            totaloutputdic[s] = tot

    return outputdic, totaloutputdic


In [10]:
sectors_dfs=process_date(Sector_dfs,Sector_total_dfs,"Year")
segments_dfs=process_date(Segment_dfs,Segment_total_dfs,"Year")
subsegments_dfs=process_date(SubSegment_dfs,SubSegment_total_dfs,"Year")
subcategories_dfs=process_date(SubCategory_dfs,SubCategory_total_dfs,"Year")


In [11]:
if len(sectors)!=0:
    modified_sectors_dfs_new = {}
    sectors_totals_new= {}
    modified_sectors_dfs_new, sectors_totals_new = secsegclean(sectors_dfs)
    #P12M
    modified_sectors_P12M_new = {}
    dya_sectors_new = {}
    modified_sectors_P12M_new, dya_sectors_new = secsegclean(Sector_P12M_dfs)
    #Client
    modified_sectors_clients_new = {}
    modified_sectors_clients_new,_ = secsegclean(Sector_client_dfs)

if len(segments)!=0:
    modified_segment_dfs_new = {}
    segment_totals_new= {}
    modified_segment_dfs_new, segment_totals_new = secsegclean(segments_dfs)
    #P12M
    modified_segment_P12M_new = {}
    dya_segment_new = {}
    modified_segment_P12M_new, dya_segment_new = secsegclean(Segment_P12M_dfs)
    #Client
    modified_segment_clients_new = {}
    modified_segment_clients_new,_ = secsegclean(Segment_client_dfs)


if len(subsegments)!=0:
    modified_subsegment_dfs_new = {}
    subsegment_totals_new= {}
    modified_subsegment_dfs_new, subsegment_totals_new = secsegclean(subsegments_dfs)
    #P12M
    modified_subsegment_P12M_new = {}
    dya_subsegment_new = {}
    modified_subsegment_P12M_new, dya_subsegment_new = secsegclean(SubSegment_P12M_dfs)
    #Client
    modified_subsegment_clients_new = {}
    modified_subsegment_clients_new,_ = secsegclean(SubSegment_client_dfs)


if len(subcategories)!=0:
    modified_subcategories_dfs_new = {}
    subcategories_totals_new= {}
    modified_subcategories_dfs_new, subcategories_totals_new = secsegclean(subcategories_dfs)
    #P12M
    modified_subcategories_P12M_new = {}
    dya_subcategories_new = {}
    modified_subcategories_P12M_new, dya_subcategories_new = secsegclean(SubCategory_P12M_dfs)
    #Client
    modified_subcategories_clients_new = {}
    modified_subcategories_clients_new,_ = secsegclean(SubCategory_client_dfs)


In [12]:
def addOther(dic,col=f"{BrandOrTopB}",TorB="Top"):
    for key, df in dic.items():
        if col in df.columns:
            others_row = df[df[col].str.contains('Others', case=False, na=False)]
            rows_wo_others = df[~df[col].str.contains('Others', case=False, na=False)]
            if not others_row.empty:
                if TorB=="Top":
                # Re-order: "Others" first, then the rest of the rows
                    dic[key] = pd.concat([others_row, rows_wo_others])    
                else:
                    dic[key] = pd.concat([ rows_wo_others,others_row])    
    return dic                    

In [13]:
manuf_dfs_new=process_date(manuf_dfs,manuf_total_dfs,"Year")
manuf_P12M_dfs_new=process_date(manuf_12_dfs,manuf_12_total_dfs,"Time Period")
brands_dfs_new=process_date(brand_dfs,brand_total_dfs,"Year")


In [14]:
modified_manuf_dfs_new = {}
modified_manuf_totals_new= {}
modified_manuf_P12M_new = {}
modified_manuf_P12M_regions_new = {}

for s in manuf_dfs_new.keys():
    t = manuf_dfs_new[s].copy()
    grand_total_values = []
    # for col in t.select_dtypes(include='number').columns:
    #     if "Price" in col:
    #         grand_total_values.append(t[col].mean())  # Take average for price columns
    #     else:
    #         grand_total_values.append(t[col].sum())   # Take sum for other columns
    # grand_total_row = pd.DataFrame([["Grand Total"] + grand_total_values], columns=t.columns)
    # t = pd.concat([t, grand_total_row], ignore_index=True)
    t.columns = t.columns.str.strip()            
    mod = t[(~t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    numeric_cols = mod.select_dtypes(include=['number']).columns
    mod = mod.loc[~(mod[numeric_cols] == 0).all(axis=1)]
    mod = mod.sort_values([col for col in mod.columns if 'Value Share' in col], ascending=False)
    tot = t[(t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    if not mod.empty:
        modified_manuf_dfs_new[s] = mod
    if not tot.empty:
        modified_manuf_totals_new[s] = tot
    

df_toDrop =[key for key in modified_manuf_dfs_new.keys() if (modified_manuf_dfs_new[key].shape[1]>2) and (modified_manuf_dfs_new[key].shape[1]<4)]

for key in df_toDrop:
    del modified_manuf_dfs_new[key]

    
emptyDf=[key for key in modified_manuf_dfs_new.keys() if modified_manuf_dfs_new[key].shape[1]<4]

for dic in [modified_manuf_dfs_new]:
    if emptyDf:
        columnsToAdd=list(set(dic.keys())-set(emptyDf))[0]
        for df in emptyDf:
            dic[df]=dic[df].drop(columns=[''], errors = 'ignore')#list(set(dic[df].columns)-set(['']))
            emptyCol=list(set(dic[columnsToAdd].columns)-set(dic[df].columns)-set(['']))
            dic[df][emptyCol]=0.0
modified_manuf_dfs_new = dfSort(modified_manuf_dfs_new, client_manuf,f'{ManufOrTopC}',10, 'Total Value Share')
addOther(modified_manuf_dfs_new,col=f'{ManufOrTopC}',TorB="Bottom")


for s in manuf_P12M_dfs_new.keys():
    t = manuf_P12M_dfs_new[s].copy()
    grand_total_values = []
    # for col in t.select_dtypes(include='number').columns:
    #     if "Price" in col:
    #         grand_total_values.append(t[col].mean())  # Take average for price columns
    #     else:
    #         grand_total_values.append(t[col].sum())   # Take sum for other columns
    # grand_total_row = pd.DataFrame([["Grand Total"] + grand_total_values], columns=t.columns)
    # t = pd.concat([t, grand_total_row], ignore_index=True)
    t.columns = t.columns.str.strip()            

    mod = t[(~t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    mod = mod.sort_values([col for col in mod.columns if 'Value Share' in col], ascending=False)
    tot = t[(t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    if not mod.empty:
        modified_manuf_P12M_new[s] = mod

df_toDrop =[key for key in modified_manuf_P12M_new.keys() if (modified_manuf_P12M_new[key].shape[1]>2) and (modified_manuf_P12M_new[key].shape[1]<4)]
for key in df_toDrop:
    del modified_manuf_P12M_new[key]


emptyDf=[key for key in modified_manuf_P12M_new.keys() if modified_manuf_P12M_new[key].shape[1]<4]
for dic in [modified_manuf_P12M_new]:
    if emptyDf:
        columnsToAdd=list(set(dic.keys())-set(emptyDf))[0]
        for df in emptyDf:
            dic[df]=dic[df].drop(columns=[''])#list(set(dic[df].columns)-set(['']))
            emptyCol=list(set(dic[columnsToAdd].columns)-set(dic[df].columns)-set(['']))
            dic[df][emptyCol]=0.0

modified_manuf_P12M_new = dfSort(modified_manuf_P12M_new, client_manuf,f'{ManufOrTopC}',10, 'Value Share')
addOther(modified_manuf_P12M_new,col=f"{ManufOrTopC}",TorB="Bottom")



{'Protein | NATIONAL':    Top Companies  Volume Sales P12M  Value Sales P12M  Value Share P12M  \
 1   Danone Group          7563908.0        49572677.0          0.493865   
 0  Private Label          7129361.0        33619973.0          0.334937   
 1     Ot. Marcas           981431.0         7968041.0          0.079381   
 2          Reina          1041202.0         5166419.0          0.051470   
 3         Nestle           529537.0         2851217.0          0.028405   
 0         Others           159870.0         1198600.0          0.011941   
 
    Volume Share P12M  Share DYA P12M  Av Price/KG P12M  Volume Sales LY  \
 1           0.434575        0.010302          6.553800        5637728.0   
 0           0.409608        0.057127          4.715700        4106566.0   
 1           0.056387       -0.033427          8.118800        1258737.0   
 2           0.059821       -0.036035          4.962000        1834342.0   
 3           0.030424       -0.001888          5.384400         

In [15]:
def dfSort(df, client_manuf, col, num=5, salesCol=''):
    """
    Sort and filter dataframes based on specified conditions.

    Parameters:
    - df (dict): Dictionary of dataframes.
    - client_manuf (list): List of client manufacturers.
    - col (str): Column to sort by.
    - num (int): Number of items to keep.
    - salesCol (str): Column indicating sales.

    Returns:
    - newDic (dict): Modified dictionary of dataframes.
    """
    newDic = {}  # Initialize an empty dictionary to store modified dataframes
    global ot  # Global variable to store 'other' data

    # Iterate through each dataframe in the dictionary
    for key in df.keys():
        if salesCol:  # Check if a sales column is specified
            sales = [column for column in df[key].columns if salesCol in column][0]  # Get the sales column name

        # Filter dataframe based on sales threshold and sort by sales
        topManuf = df[key][df[key][sales].astype(float) >= 0.01].sort_values(sales, ascending=False)

        # Exclude client manufacturers from the top manufacturers list
        topManuf = topManuf[(~topManuf[col].isin(client_manuf))].reset_index(drop=True)

        clientDf = df[key][df[key][col].isin(client_manuf)]
        clientDf = clientDf.sort_values(sales, ascending=False)

        # Define a list of common 'other' terms
        otherList = ["Other", "Other Manufacturer", "Others", "All Others"]

        # Check if any of the 'other' terms are present in the top manufacturers list
        if any(otherValue in topManuf.iloc[:num - len(clientDf)][col].unique() for otherValue in otherList):
            topManuf = topManuf.iloc[:num - len(clientDf)]  # Filter out 'other' items
            topManuf = topManuf[~topManuf[col].str.contains("Other|Others|Other Manufacturer|All Others")]

        else:
            topManuf = topManuf.iloc[:num - len(clientDf) - 1]  # Exclude one extra row

        # Extract rows containing 'other' manufacturers
        other_rows = df[key][df[key][col].str.contains("Other|Others|Other Manufacturer|All Others")]

        # Sort client-specific data by sales

        try:
            total = pd.DataFrame()  # Initialize an empty dataframe to store 'other' data

            # Extract 'other' data from the dataframe
            other = df[key][(~df[key][col].isin(topManuf[col])) & (~df[key][col].isin(clientDf[col])) & (~df[key][col].isin(other_rows[col]))]
            other = pd.concat([other, other_rows])
            # Calculate mean and sum of 'other' data
            if other.shape[0] != 0:
                ot = other
                otMean = ot[[c for c in ot.columns if 'Price' in c]]
                otsum = ot[[c for c in ot.columns if 'Price' not in c]].drop(columns=col)
                otMean = pd.DataFrame(otMean.mean().values.reshape(1, -1), columns=otMean.columns)
                otsum = pd.DataFrame(otsum.sum().values.reshape(1, -1), columns=otsum.columns)
                otsum[col] = 'Others'
                otMean[col] = 'Others'
                total = otMean.merge(otsum)

        except:
            total = pd.DataFrame()  # In case of any exceptions, initialize an empty dataframe

        # Concatenate and sort dataframes, drop duplicates, and update the dictionary
        newDic[key] = pd.concat([topManuf, total]).sort_values(sales, ascending=False).drop_duplicates(subset=col)
        if len(clientDf)<num:
            newDic[key] = pd.concat([clientDf, newDic[key]])
        else:
    
            newDic[key] = clientDf
        newDic[key] = newDic[key].sort_values(sales, ascending = False)

    return newDic  # Return the modified dictionary of dataframes


In [16]:
modified_brands_share_new = {}
modified_brands_totals_new = {}
modified_brands_P12M_new = {}
modified_brands_evolution_new = {}
modified_manuf_evolution_new = {}


for s in brands_dfs_new.keys():
    t = brands_dfs_new[s].copy()
    t.columns = t.columns.str.strip()            
    grand_total_values = []
    # for col in t.select_dtypes(include='number').columns:
    #     if "Price" in col:
    #         grand_total_values.append(t[col].mean())  # Take average for price columns
    #     else:
    #         grand_total_values.append(t[col].sum())   # Take sum for other columns
    # grand_total_row = pd.DataFrame([["Grand Total"] + grand_total_values], columns=t.columns)
    # t = pd.concat([t, grand_total_row], ignore_index=True)
    
    mod = t[(~t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    mod = mod.sort_values([col for col in mod.columns if 'Value Share' in col], ascending=False)
    tot = t[(t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    if not mod.empty:
        modified_brands_share_new[s] = mod
    if not tot.empty:
        modified_brands_totals_new[s] = tot


df_toDrop =[key for key in modified_brands_share_new.keys() if (modified_brands_share_new[key].shape[1]>2) and (modified_brands_share_new[key].shape[1]<4)]
for key in df_toDrop:
    del modified_brands_share_new[key]


emptyDf=[key for key in modified_brands_share_new.keys() if modified_brands_share_new[key].shape[1]<4]
for dic in [modified_brands_share_new]:
    if emptyDf:
        columnsToAdd=list(set(dic.keys())-set(emptyDf))[0]
        for df in emptyDf:
            dic[df]=dic[df].drop(columns=[''])#list(set(dic[df].columns)-set(['']))
            emptyCol=list(set(dic[columnsToAdd].columns)-set(dic[df].columns)-set(['']))
            dic[df][emptyCol]=0.0
p = 10 if len(client_brands) < 10 else len(client_brands) + 1

modified_brands_share_new = dfSort(modified_brands_share_new, client_brands,f'{BrandOrTopB}',p, 'Total Value Share')
for key in modified_brands_share_new.keys():
    modified_brands_share_new[key] = modified_brands_share_new[key][modified_brands_share_new[key]['Total Value Share'] != 0]
# Move rows containing "Others" in 'Top Brands' column to the first row
addOther(modified_brands_share_new,col=f'{BrandOrTopB}',TorB="Bottom")

for s in brand_12_dfs.keys():
    t = brand_12_dfs[s].copy()
    grand_total_values = []
    # for col in t.columns[1:]:  # Exclude the first column
    #     if "Price" in col:
    #         grand_total_values.append(t[col].mean())  # Take average for price columns
    #     else:
    #         grand_total_values.append(t[col].sum())   # Take sum for other columns
    # grand_total_row = pd.DataFrame([["Grand Total"] + grand_total_values], columns=t.columns)
    # t = pd.concat([t, grand_total_row], ignore_index=True)
    t.columns = t.columns.str.strip()            
    mod = t[(~t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    mod = mod.sort_values([col for col in mod.columns if 'Value Share' in col], ascending=False)
    tot = t[(t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    modified_brands_P12M_new[s] = mod

df_toDrop =[key for key in modified_brands_P12M_new.keys() if (modified_brands_P12M_new[key].shape[1]>2) and (modified_brands_P12M_new[key].shape[1]<4)]
for key in df_toDrop:
    del modified_brands_P12M_new[key]

emptyDf=[key for key in modified_brands_P12M_new.keys() if modified_brands_P12M_new[key].shape[1]<4]
for dic in [modified_brands_P12M_new]:
    if emptyDf:
        columnsToAdd=list(set(dic.keys())-set(emptyDf))[0]
        for df in emptyDf:
            dic[df]=dic[df].drop(columns=[''])#list(set(dic[df].columns)-set(['']))
            emptyCol=list(set(dic[columnsToAdd].columns)-set(dic[df].columns)-set(['']))
            dic[df][emptyCol]=0.0

modified_brands_P12M_new = dfSort(modified_brands_P12M_new, client_brands,f'{BrandOrTopB}',10, 'Value Share')
addOther(modified_brands_P12M_new,col=f'{BrandOrTopB}',TorB="Bottom")

    

{'Bifidus | NATIONAL':        Top Brands  Volume Sales  Value Sales  Value Share  Volume Share  \
 10        Activia    47814713.0  225717990.0     0.759056      0.627564   
 0   Private Label    26539734.0   65021911.0     0.218659      0.348332   
 0          Others     1836508.0    6626739.0     0.022285      0.024104   
 
     Share DYA  Av Price/KG  IYA Price/KG  
 10  -0.004184     4.720700      0.986000  
 0    0.004277     2.450000      0.997000  
 0   -0.000093     5.835127      1.015636  ,
 'Cholesterol | NATIONAL':       Top Brands  Volume Sales  Value Sales   Value Share  Volume Share  \
 3        Danacol    16593358.0   99795278.0  8.871473e-01  8.581719e-01   
 0  Private Label     2030436.0    6504924.0  5.782664e-02  1.050097e-01   
 1          Kaiku      711907.0    6189867.0  5.502589e-02  3.681826e-02   
 0         Others           4.0         18.0  1.600141e-07  2.068712e-07   
 
    Share DYA  Av Price/KG  IYA Price/KG  
 3   0.010026       6.0142         0.987  
 

In [17]:
def evolution_cleaning(dic,colname="Top Brands"):
    modified_evolution_new={}
    modified_evolution_sorted_new = {}
    for s in dic.keys():
        t = dic[s].copy()
        t.columns = t.columns.str.strip()            
        t = t.applymap(lambda x: float(x) if pd.to_numeric(x, errors='coerce') == x else x)
        t = t.applymap(lambda x: 0 if pd.isna(x) else x)
        mod = t[(~t[t.columns[0]].astype(str).str.contains('Grand Total'))]
        mod = mod.sort_values([col for col in mod.columns if 'Value Share' in col], ascending=False)
        tot = t[(t[t.columns[0]].astype(str).str.contains('Grand Total'))]
        if not mod.empty:
            modified_evolution_new[s] = mod
    for k in modified_evolution_new.keys():
        x = modified_evolution_new[k].copy()
        x[colname] = np.where(x[colname]==0, x['Year'], x[colname])
        x[colname] = x[colname].str.replace('2022 ','').str.replace('2021 ','').str.replace('2023 ',"").str.replace('2020 ','').str.replace('2024 ','').str.replace('2025 ','')
        x = x[x[colname] != 'Others']
        x = x[x[colname] != 'All Others']
        x = x[x['Price Evolution'] != 0]
        brand_sales = x.groupby(colname)['Value Sales'].sum().sort_values(ascending=False).head(6)
        if colname==f'BrandOrTopB':
            final = x[(x[colname].isin(pd.Series(brand_sales.index))) | (x[colname].isin(client_brands))]
        else:
            final = x[(x[colname].isin(pd.Series(brand_sales.index))) | (x[colname].isin(client_manuf))]
        final[['Price Evolution', 'Share Evolution']] = round(final[['Price Evolution', 'Share Evolution']].replace(np.nan,0).astype(float)*100).astype(int)
        final = final.drop_duplicates(subset = ["Year", colname])
        final = final.sort_values('Year')  
        final["Year"]=final["Year"].astype(int)      
        modified_evolution_sorted_new[k] = final
    keys_to_remove = []
    for key, df in modified_evolution_sorted_new.items():
        if df.empty:  # Remove the () from df.empty
            keys_to_remove.append(key)
 
    for key in keys_to_remove:
        del modified_evolution_sorted_new[key]
 
    return modified_evolution_sorted_new

In [18]:
def add_tot(dic, totdic):
    result_dic = {}  # Store results for each key
    for k in dic.keys():
        df = dic[k].copy()
        dt = totdic[k].copy()
        final_rows = []
        for year, group in df.groupby("Year"):
            final_rows.append(group)  # Append the original group
            total_row = dt[dt["Year"] == year]
            if not total_row.empty:
                total_row = total_row.copy()
                total_row["Year"] = f"{year} Total"  # Set as "Total <Year>"
                final_rows.append(total_row)  # Append the modified total row
        final_df = pd.concat(final_rows, ignore_index=True)
        result_dic[k] = final_df  # Store the result for each key
    return result_dic  # Return a dictionary of processed DataFrames


In [19]:
modified_brands_evolution_sorted=evolution_cleaning(brands_evolution,colname=f'{BrandOrTopB}')
modified_manuf_evolution_sorted=evolution_cleaning(manuf_evolution,colname=f'{ManufOrTopC}')
modified_brands_evolution_sorted_new=add_tot(modified_brands_evolution_sorted,brands_evolution_total)
modified_manuf_evolution_sorted_new=add_tot(modified_manuf_evolution_sorted,brands_evolution_total)


In [20]:
def grandtot(dic,marketlis):
    cleaned_dic = {}  # Store cleaned DataFrames without "Grand Total"
    total_dic = {}  # Store DataFrames with only "Grand Total"
    for k in dic.keys():
        df = dic[k].copy()
        total_df = df[df[df.columns[0]] == "Grand Total"]
        cleaned_df = df[df[df.columns[0]] != "Grand Total"]
        cleaned_df = cleaned_df[cleaned_df[cleaned_df.columns[0]].isin(marketlis)]
        if not cleaned_df.empty:
            cleaned_dic[k] = cleaned_df
        if not total_df.empty:
            total_dic[k] = total_df

    return cleaned_dic, total_dic


In [21]:
# Ensure final dictionaries are initialized
modified_retailer_regions = {}
modified_retailer_channels = {}
modified_retailer_markets = {}

modified_channels_regions = {}
modified_channels_channels = {}
modified_channels_markets = {}

modified_cust_regions = {}
modified_cust_channels = {}
modified_cust_markets = {}
dya_retailer={}
dya_channel={}
dya_cust = {}
# ********** Retailer *********
if len(retailers) != 0:
    modified_retailer_regions,dya_retailer = grandtot(globals().get("modified_retailer_Region", {}) or {},regions_RET)
    modified_retailer_channels,_ =grandtot( globals().get("modified_retailer_Channel", {}) or {},channels_RET)
    modified_retailer_markets,_ = grandtot(globals().get("modified_retailer_Market", {}) or {},market_RET)

# ********* Channels **********
if len(channels) != 0:
    modified_channels_regions,dya_channel = grandtot(globals().get("modified_channels_Region", {}) or {},regions_CHAN)
    modified_channels_channels,_ = grandtot(globals().get("modified_channels_Channel", {}) or {},channels_CHAN)
    modified_channels_markets,_ = grandtot(globals().get("modified_channels_Market", {}) or {},market_CHAN)




# ******** POS ********
if len(cust) != 0: 
    modified_cust_regions,dya_cust = grandtot(globals().get("modified_cust_Region", {}) or {},regions_CUST)
    modified_cust_channels,_ = grandtot(globals().get("modified_cust_Channel", {}) or {},channels_CUST)
    modified_cust_markets,_ = grandtot(globals().get("modified_cust_Market", {}) or {},market_CUST)


In [22]:
#### Client
modified_retailer_regions_client = {}
modified_retailer_channels_client = {}
modified_retailer_markets_client = {}

modified_channels_regions_client = {}
modified_channels_channels_client = {}
modified_channels_markets_client = {}

modified_cust_regions_client = {}
modified_cust_channels_client = {}
modified_cust_markets_client = {}
#********** Retailer *********
if len(retailers) != 0:
    modified_retailer_regions_client,_ = grandtot(globals().get("modified_retailer_Region_client", {}) or {},regions_RET)
    modified_retailer_channels_client,_ = grandtot(globals().get("modified_retailer_Channel_client", {}) or {},channels_RET)
    modified_retailer_markets_client,_ = grandtot(globals().get("modified_retailer_Market_client", {}) or {},market_RET)

# ********* Channels **********
if len(channels) != 0:
    modified_channels_regions_client,_ = grandtot(globals().get("modified_channels_Region_client", {}) or {},regions_CHAN)
    modified_channels_channels_client,_ = grandtot(globals().get("modified_channels_Channel_client", {}) or {},channels_CHAN)
    modified_channels_markets_client,_ =grandtot(globals().get("modified_channels_Market_client", {}) or {},market_CHAN)

# ******** POS ********
if len(cust) != 0: 
    modified_cust_regions_client,_ = grandtot(globals().get("modified_cust_Region_client", {}) or {},regions_CUST)
    modified_cust_channels_client,_ = grandtot(globals().get("modified_cust_Channel_client", {}) or {},channels_CUST)
    modified_cust_markets_client,_ = grandtot(globals().get("modified_cust_Market_client", {}) or {},market_CUST)



In [23]:
modified_calendar_new = {}
for table in calendar_dfs.keys():
    modified_tables = []
    cleantable = calendar_dfs[table].copy()
    cleantable =cleantable[cleantable['MonthYear']!='Grand Total']
    cleantable['time'] = pd.to_datetime(cleantable['MonthYear'], format='%b-%y')
    cleantable = cleantable.nlargest(25, 'time')
    cleantable = cleantable.sort_values('time')
    cleantable['Av Price/KG'] = cleantable['Av Price/KG'].replace(np.nan, 0)
    cleantable['Av Price/KG'] =round(cleantable['Av Price/KG'].astype(float),decimals)
    modified_calendar_new[table] = cleantable


keys_to_remove = []
for key, df in modified_calendar_new.items():
    if df.empty:  # Remove the () from df.empty
        keys_to_remove.append(key)

for key in keys_to_remove:
    del modified_calendar_new[key]


In [24]:
modified_categories_dfs_new=[]
for key in categories_overview_dfs.keys():
    try:
        dfCat=categories_overview_dfs[key].copy()
        dfCat[['Cat Volume Sales/M','Cat Value Sales/M']]=round(dfCat[['Volume Sales','Value Sales']].astype(float)/10**6, 2)
        dfCat[['Cat Volume Sales Change YA','Cat Value Sales Change YA']]=round((dfCat[['Volume Sales IYA','Value Sales IYA']].astype(float)*100)-100,2)
        dfCat['Cat IYA Price/KG']=round((dfCat['IYA Price/KG'].astype(float)*100),decimals)
        dfCat['flag']=1

        dfMamu=categories_overview_manuf_dfs[key].copy()
        
        dfMamu[['Manu Volume Sales/M','Manu Value Sales/M']]=round(dfMamu[['Volume Sales','Value Sales']].astype(float)/10**6, 2)
        dfMamu[['Manu Volume Sales Change YA','Manu Value Sales Change YA']]=round((dfMamu[['Volume Sales IYA','Value Sales IYA']].astype(float)*100)-100,2)
        dfMamu['Manu IYA Price/KG']=round((dfMamu['IYA Price/KG'].astype(float)*100),2)
        dfMamu['Impact Growth'] = dfMamu['Manu Value Sales Change YA'] - dfMamu['Manu Volume Sales Change YA']
        dfMamu['flag']=1

        table = pd.merge(dfCat[['flag','Cat Volume Sales/M','Cat Value Sales/M','Cat Volume Sales Change YA','Cat Value Sales Change YA','Cat IYA Price/KG']],dfMamu[['flag','Manu Volume Sales/M','Manu Value Sales/M','Manu Volume Sales Change YA','Manu Value Sales Change YA','Manu IYA Price/KG','Impact Growth']]).drop(columns='flag')
        table = table[['Cat Value Sales/M','Cat Value Sales Change YA','Cat IYA Price/KG', 'Manu Value Sales/M','Manu Value Sales Change YA','Manu IYA Price/KG','Cat Volume Sales/M','Cat Volume Sales Change YA','Manu Volume Sales/M','Manu Volume Sales Change YA','Impact Growth']]
        table['Price Impact'] = str(int(round(table['Manu Value Sales Change YA']))-int(round(table['Manu Volume Sales Change YA'])))+"pts"
        
        table['Cat Value Sales Change YA']=str(int(round(table['Cat Value Sales Change YA'])))+'%'
        table['Cat Volume Sales Change YA']=str(int(round(table['Cat Volume Sales Change YA'])))+'%'
        table['Manu Value Sales Change YA']=str(int(round(table['Manu Value Sales Change YA'])))+'%'
        table['Manu Volume Sales Change YA']=str(int(round(table['Manu Volume Sales Change YA'])))+'%'
        table['Cat IYA Price/KG']=str(int(round(table['Cat IYA Price/KG'])))+'%'
        table['Manu IYA Price/KG']=str(int(round(table['Manu IYA Price/KG'])))+'%'
        lis=['Cat Value Sales/M','Manu Value Sales/M','Cat Volume Sales/M','Manu Volume Sales/M','Impact Growth']
        for col in lis:
            if round(table[col][0],1)==int(round(table[col][0],1)):
                table[col]=int(round(table[col]))
            else:
                table[col]=round(table[col],1)
        

        dfVal = categories_values_dfs[key].copy()
        first_col = dfVal.columns[0]
        dfValnogr = dfVal[dfVal[first_col] != "Grand Total"]
        grand_total = dfVal[dfVal[first_col] == "Grand Total"]
        ordered_rows = []
        for month in dfValnogr[first_col].unique():
            group = dfValnogr[dfValnogr[first_col] == month]
            total_row = group.sum(numeric_only=True)
            total_row[first_col] = f"{month} Total"  # Append "Total" to the group name
            total_row = pd.DataFrame([total_row])  # Convert to DataFrame
            ordered_rows.append(group)  # Add the group rows
            ordered_rows.append(total_row)  # Add the total row
        df_with_totals = pd.concat(ordered_rows, ignore_index=True)
        df_final = pd.concat([df_with_totals, grand_total], ignore_index=True)
        modified_categories_dfs_new.append([table, df_final, key])

    # except:
        # print(key)
    except Exception as e:
        print(f"Error with key '{key}': {e}")
    

In [25]:
sharGrowthDf_sec={}
sharGrowthDf_seg={}
sharGrowthDf_subseg={}
sharGrowthDf_subcat={}
shareGrowthDf_retailers_REG={}
shareGrowthDf_retailers_CHAN={}
shareGrowthDf_retailers_MAR={}
shareGrowthDf_channels_REG={}
shareGrowthDf_channels_CHAN={}
shareGrowthDf_channels_MAR={}
shareGrowthDf_custom_REG={}
shareGrowthDf_custom_CHAN={}
shareGrowthDf_custom_MAR={}

for key in categories + sectors + segments + subsegments +subcategories:
    
    shareGrowthDf_retailers_REG[key]=[k.split(' | ')[1] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in regions_RET ] 
    shareGrowthDf_retailers_CHAN[key]=[k.split(' | ')[1] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in channels_RET ]
    shareGrowthDf_retailers_MAR[key]=[k.split(' | ')[1] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in market_RET ] 
    
    shareGrowthDf_channels_REG[key]=[k.split(' | ')[1] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in regions_CHAN ] 
    shareGrowthDf_channels_CHAN[key]=[k.split(' | ')[1] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in channels_CHAN ]
    shareGrowthDf_channels_MAR[key]=[k.split(' | ')[1] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in market_CHAN ]

    shareGrowthDf_custom_REG[key]=[k.split(' | ')[1] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in regions_CUST ] 
    shareGrowthDf_custom_CHAN[key]=[k.split(' | ')[1] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in channels_CUST ]
    shareGrowthDf_custom_MAR[key]=[k.split(' | ')[1] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in market_CUST ]

    if len(shareGrowthDf_retailers_REG[key])==0:
        shareGrowthDf_retailers_REG.pop(key,None)    
    if len(shareGrowthDf_retailers_CHAN[key])==0:
        shareGrowthDf_retailers_CHAN.pop(key,None)
    if len(shareGrowthDf_retailers_MAR[key])==0:
        shareGrowthDf_retailers_MAR.pop(key,None)
    
    if len(shareGrowthDf_channels_REG[key])==0:
        shareGrowthDf_channels_REG.pop(key,None)    
    if len(shareGrowthDf_channels_CHAN[key])==0:
        shareGrowthDf_channels_CHAN.pop(key,None)
    if len(shareGrowthDf_channels_MAR[key])==0:
        shareGrowthDf_channels_MAR.pop(key,None)

    if len(shareGrowthDf_custom_REG[key])==0:
        shareGrowthDf_custom_REG.pop(key,None)    
    if len(shareGrowthDf_custom_CHAN[key])==0:
        shareGrowthDf_custom_CHAN.pop(key,None)
    if len(shareGrowthDf_custom_MAR[key])==0:
        shareGrowthDf_custom_MAR.pop(key,None)
    

for key in ["NATIONAL"] + regions_RET + channels_RET+market_RET +  regions_CHAN +channels_CHAN+ market_CHAN+regions_CUST+channels_CUST+ market_CUST:
    sharGrowthDf_sec[key]=[k.split(' | ')[0] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[1] and k.split(' | ')[0] in sectors ]
    sharGrowthDf_seg[key]=[k.split(' | ')[0] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[1] and k.split(' | ')[0] in segments]
    sharGrowthDf_subseg[key]=[k.split(' | ')[0] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[1] and k.split(' | ')[0] in subsegments]
    sharGrowthDf_subcat[key]=[k.split(' | ')[0] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[1] and k.split(' | ')[0] in subcategories]
    
    if len(sharGrowthDf_sec[key])==0:
        sharGrowthDf_sec.pop(key,None)
        
    if len(sharGrowthDf_seg[key])==0:
        sharGrowthDf_seg.pop(key,None)
    
    if len(sharGrowthDf_subseg[key])==0:
        sharGrowthDf_subseg.pop(key,None)
    
    if len(sharGrowthDf_subcat[key])==0:
        sharGrowthDf_subcat.pop(key,None)

### Create Market Growth Market List for duplication and replacement

In [26]:
markets_name = ['regions_RET' , 'channels_RET','market_RET','regions_CHAN' ,'channels_CHAN', 'market_CHAN' ,'regions_CUST' ,'channels_CUST', 'market_CUST']
markets_val = [regions_RET , channels_RET,market_RET  ,regions_CHAN ,channels_CHAN, market_CHAN ,regions_CUST ,channels_CUST, market_CUST]
markets_dfs = [modified_retailer_regions , modified_retailer_channels,modified_retailer_markets  ,modified_channels_regions ,modified_channels_channels,  modified_channels_markets,modified_cust_regions ,modified_cust_channels, modified_cust_markets]
market_total = [dya_retailer,dya_channel,dya_cust]
slide_by = ['Region','Channel','Market']*3
slide_for = [*['Retailer']*3,*['Channel']*3,*[customareas]*3]
market_list_Growth={}
duplication_num = []
section_name_Growth = []
for idx,val in enumerate(markets_val):
    if len(val)>0:
        market_list_Growth[markets_name[idx]]=[val,markets_dfs[idx],market_total[idx//3],slide_by[idx],slide_for[idx]]
        duplication_num.append(len(markets_dfs[idx].keys()))
        section_name_Growth.append('Market Growth By '+slide_for[idx]+' For '+slide_by[idx])
section_number = len(market_list_Growth.keys())


#####  Create Value_AvgPrice Market List for duplication and replacement

In [27]:
markets_name = ['regions_RET' , 'channels_RET','market_RET'  ,'regions_CHAN' ,'channels_CHAN', 'market_CHAN' ,'regions_CUST' ,'channels_CUST', 'market_CUST']
markets_val = [regions_RET , channels_RET,market_RET  ,regions_CHAN ,channels_CHAN, market_CHAN ,regions_CUST ,channels_CUST, market_CUST]
markets_dfs = [modified_retailer_regions , modified_retailer_channels,modified_retailer_markets  ,modified_channels_regions ,modified_channels_channels,  modified_channels_markets,modified_cust_regions ,modified_cust_channels, modified_cust_markets]
client_dfs = [modified_retailer_regions_client , modified_retailer_channels_client,modified_retailer_markets_client  ,modified_channels_regions_client ,modified_channels_channels_client,  modified_channels_markets_client,modified_cust_regions_client ,modified_cust_channels_client, modified_cust_markets_client]
market_total = [dya_retailer,dya_channel,dya_cust]
slide_by = ['Region','Channel','Market']*3
slide_for = [*['Retailer']*3,*['Channel']*3,*[customareas]*3]
market_list_Avg={}
duplication_num_Avg = []
section_name_Avg = []


for idx,val in enumerate(markets_val):
    if len(val)>0:
        market_list_Avg[markets_name[idx]]=[val,markets_dfs[idx],client_dfs[idx],slide_by[idx],slide_for[idx]]
        duplication_num_Avg.append(len(client_dfs[idx].keys()))
        section_name_Avg.append('Value Vs AvgPrice By '+slide_for[idx]+' For '+slide_by[idx])
section_number_Avg = len(market_list_Avg.keys())

### Slide 12 : Category Evolution Analysis 

In [28]:
def cleanSectorSegment(dfDic):
    catDic, totalDic = {}, {}
    for key, df in dfDic.items():
        df = df.copy()
        catName=df.columns[0]
        df = df.applymap(lambda x: float(x) if pd.to_numeric(x, errors='coerce') == x else x)
        df = df.applymap(lambda x: 0 if pd.isna(x) else x)

        df[catName] = df[catName].astype(str)
        addative = df[~df[catName].str.contains(' Total')].sort_values('WoB %', ascending=False)

        total = df[df[catName].str.contains(' Total')]
        total[catName] = total[catName].str.replace(' Total', '')

        # Check if the total DataFrame is empty and if so, add a "Grand" total row with zeros
        if total.empty:
            zero_row = pd.Series([0] * (len(df.columns) - 1), index=df.columns[1:])
            zero_row[catName] = "Grand"
            total = pd.concat([total, zero_row.to_frame().T], ignore_index=True)
        if not addative.empty:
            catDic[key.replace('  ', ' ')] = addative
        if not total.empty:
            totalDic[key.replace('  ', ' ')] = total

    return catDic, totalDic


In [29]:
cea_sector_P12M_new,cea_dya_sector_new,cea_sector_clients_new,cea_sector_clients_total_new ={},{},{},{}

if sectors:
    cea_sectors_P12M_new,cea_dya_sectors_new=cleanSectorSegment(Sector_P12M_dfs)
    cea_sectors_clients_new,cea_sectors_clients_total_new=cleanSectorSegment(Sector_client_dfs_category)

cea_segment_P12M_new,cea_dya_segment_new,cea_segment_clients_new,cea_segment_clients_total_new ={},{},{},{}

if segments:
    cea_segment_P12M_new,cea_dya_segment_new=cleanSectorSegment(Segment_P12M_dfs)
    cea_segment_clients_new,cea_segment_clients_total_new=cleanSectorSegment(Segment_client_dfs_category)

In [30]:
cea_subsegment_P12M_new,cea_dya_subsegment_new,cea_subsegment_clients_new,cea_subsegment_clients_total_new ={},{},{},{}
if subsegments:
    cea_subsegment_P12M_new,cea_dya_subsegment_new=cleanSectorSegment(SubSegment_P12M_dfs)
    cea_subsegment_clients_new,cea_subsegment_clients_total_new=cleanSectorSegment(SubSegment_client_dfs_category)

cea_subcategory_P12M_new,cea_dya_subcategory_new,cea_subcategory_clients_new,cea_subcategory_clients_total_new ={},{},{},{}
if subcategories:
    cea_subcategory_P12M_new,cea_dya_subcategory_new=cleanSectorSegment(SubCategory_P12M_dfs)
    cea_subcategory_clients_new,cea_subcategory_clients_total_new=cleanSectorSegment(SubCategory_client_dfs_category)

## Slide 13 : Share And Growth

In [31]:
# Ensure final dictionaries are initialized
retailerRegion = {}
retailerChannel = {}
retailertMarket = {}

channelRegion = {}
channelChannel = {}
channelMarket = {}

custRegion = {}
custChannel = {}
custMarket = {}

retailerTotal={}
channelTotal={}
custTotal={}
#********** Retailer *********
if len(retailers) != 0:
    retailerRegion,retailerTotal = grandtot(globals().get("modified_retailer_Region", {}) or {},regions_RET)
    retailerChannel ,_= grandtot(globals().get("modified_retailer_Channel", {}) or {},channels_RET)
    retailertMarket,_ =grandtot(globals().get("modified_retailer_Market", {}) or {},market_RET)
# ********* Channels **********
if len(channels) != 0:
    channelRegion,channelTotal = grandtot(globals().get("modified_channels_Region", {}) or {},regions_CHAN)
    channelChannel,_ = grandtot(globals().get("modified_channels_Channel", {}) or {},channels_CHAN)
    channelMarket,_ = grandtot(globals().get("modified_channels_Market", {}) or {},market_CHAN)
# ******** POS ********
if len(cust) != 0: 
    custRegion,custTotal = grandtot(globals().get("modified_cust_Region", {}) or {},regions_CUST)
    custChannel,_ =grandtot(globals().get("modified_cust_Channel", {}) or {},channels_CUST)
    custMarket,_ = grandtot(globals().get("modified_cust_Market", {}) or {},market_CUST)



In [32]:
# Ensure final dictionaries are initialized
clientRetailerRegion = {}
clientretailerChannel = {}
clientRetailertMarket = {}

clientChannelRegion = {}
clientChannelChannel = {}
clientChannelMarket = {}

clientCustRegion = {}
clientCustChannel = {}
clientCustMarket = {}

retailerClientTotal={}
channelClientTotal={}
custClientTotal={}
#********** Retailer *********
if len(retailers) != 0:
    clientRetailerRegion,retailerClientTotal = grandtot(globals().get("modified_retailer_Region_client", {}) or {},regions_RET)
    clientretailerChannel,_ =grandtot(globals().get("modified_retailer_Channel_client", {}) or {},channels_RET)
    clientRetailertMarket,_ = grandtot(globals().get("modified_retailer_Market_client", {}) or {},market_RET)
# ********* Channels **********
if len(channels) != 0:
    clientChannelRegion,channelClientTotal = grandtot(globals().get("modified_channels_Region_client", {}) or {},regions_CHAN)
    clientChannelChannel,_ = grandtot(globals().get("modified_channels_Channel_client", {}) or {},channels_CHAN)
    clientChannelMarket,_ = grandtot(globals().get("modified_channels_Market_client", {}) or {},market_CHAN)
# ******** POS ********
if len(cust) != 0: 
    clientCustRegion,custClientTotal =grandtot(globals().get("modified_cust_Region_client", {}) or {},regions_CUST)
    clientCustChannel,_ = grandtot(globals().get("modified_cust_Channel_client", {}) or {},channels_CUST)
    clientCustMarket,_ = grandtot(globals().get("modified_cust_Market_client", {}) or {},market_CUST)
 


In [33]:
retailerCatDic,retailerSecDic,retailerSegDic,retailerSubSegDic,retailerSubCatDic=[],[],[],[],[]
channelCatDic,channelSecDic,channelSegDic,channelSubSegDic,channelSubCatDic=[],[],[],[],[]
custCatDic,custSecDic,custSegDic,custSubSegDic,custSubCatDic=[],[],[],[],[]

# if modified_retailer_Region:

retailerCatDic = [key for key in retailerClientTotal.keys() if (len(set(categories).intersection(set(key.split(' | '))))) ]
retailerSecDic = [key for key in retailerClientTotal.keys() if len(set(sectors).intersection(set(key.split(' | '))))]
retailerSegDic = [key for key in retailerClientTotal.keys() if len(set(segments).intersection(set(key.split(' | '))))]
retailerSubSegDic = [key for key in retailerClientTotal.keys() if len(set(subsegments).intersection(set(key.split(' | '))))]
retailerSubCatDic = [key for key in retailerClientTotal.keys() if len(set(subcategories).intersection(set(key.split(' | '))))]



# if modified_channels_Region:

channelCatDic = [key for key in channelClientTotal.keys() if len(set(categories).intersection(set(key.split(' | '))))]
channelSecDic = [key for key in channelClientTotal.keys() if len(set(sectors).intersection(set(key.split(' | '))))]
channelSegDic = [key for key in channelClientTotal.keys() if len(set(segments).intersection(set(key.split(' | '))))]
channelSubSegDic = [key for key in channelClientTotal.keys() if len(set(subsegments).intersection(set(key.split(' | '))))]
channelSubCatDic = [key for key in channelClientTotal.keys() if len(set(subcategories).intersection(set(key.split(' | '))))]

# if cust_dfs_new:

custCatDic = [key for key in custClientTotal.keys() if len(set(categories).intersection(set(key.split(' | ')))) ]
custSecDic = [key for key in custClientTotal.keys() if len(set(sectors).intersection(set(key.split(' | ')))) ]
custSegDic = [key for key in custClientTotal.keys() if len(set(segments).intersection(set(key.split(' | '))))]
custSubSegDic = [key for key in custClientTotal.keys() if len(set(subsegments).intersection(set(key.split(' | '))))]
custSubCatDic = [key for key in custClientTotal.keys() if len(set(subcategories).intersection(set(key.split(' | '))))]



In [34]:
retailerField = {'Region':[retailerRegion,clientRetailerRegion,retailerTotal,retailerClientTotal],'Channel':[retailerChannel,clientretailerChannel,retailerTotal,retailerClientTotal],'Market':[retailertMarket,clientRetailertMarket,retailerTotal,retailerClientTotal]}
channelField = {'Region':[channelRegion,clientChannelRegion,channelTotal,channelClientTotal],'Channel':[channelChannel,clientChannelChannel,channelTotal,channelClientTotal],'Market':[channelMarket,clientChannelMarket,channelTotal,channelClientTotal]}
custField = {'Region':[custRegion,clientCustRegion,custTotal,custClientTotal],'Channel':[custChannel,clientCustChannel,custTotal,custClientTotal],'Market':[custMarket,clientCustMarket,custTotal,custClientTotal]}

### Slide 14: Momentum Analysis

In [35]:
def merge_dicts(target, *sources):
    for source in sources:
        if source:  # Ensure the source is not None or empty
            target.update(source)

modified_momentum_retailers_new = {}
merge_dicts(
    modified_momentum_retailers_new, 
    globals().get('modified_momentum_retailer_Region', {}), 
    globals().get('modified_momentum_retailer_Channel', {}), 
    globals().get('modified_momentum_retailer_Market', {})
)

modified_momentum_channels_new = {}
merge_dicts(
    modified_momentum_channels_new, 
    globals().get('modified_momentum_channels_Region', {}), 
    globals().get('modified_momentum_channels_Channel', {}), 
    globals().get('modified_momentum_channels_Market', {})
)

modified_momentum_cust_new = {}
merge_dicts(
    modified_momentum_cust_new, 
    globals().get('modified_momentum_cust_Region', {}), 
    globals().get('modified_momentum_cust_Channel', {}), 
    globals().get('modified_momentum_cust_Market', {})
)


In [36]:
def process_dateMomentum(manuf_dfs, datacolumn, numskpcol,hirlis=[],market=False):
    output = {}  
    for k, df in manuf_dfs.items():
        df = df.copy()
        if not df.empty:
            # Ensure numskpcol is valid
            if numskpcol not in [2, 4]:
                raise ValueError("numskpcol must be either 2 or 4")
            
            # Extract the first numskpcol columns as index
            index_cols = list(df.columns[:numskpcol])
            
            # Pivot the DataFrame
            df_pivot = df.pivot_table(index=index_cols, 
                                    columns=datacolumn, 
                                    aggfunc='sum', 
                                    dropna=False)
            df_pivot.columns = [f"{metric} {year}" for metric, year in df_pivot.columns]
            df_pivot.reset_index(inplace=True)
            df_pivot = df_pivot.dropna(subset=df_pivot.columns[numskpcol:], how='all')
            numeric_cols = df_pivot.select_dtypes(include=['number']).columns
            years = sorted(df[datacolumn].dropna().unique(), key=list(df[datacolumn].dropna()).index)
            metrics = sorted(
                set(col.rsplit(" ", 1)[0] for col in numeric_cols),
                key=lambda x: list(df.columns).index(x) if x in df.columns else float('inf')
            )
            sorted_columns = index_cols + [f"{metric} {year}" for year in years for metric in metrics]
            df_pivot = df_pivot[[col for col in sorted_columns if col in df_pivot.columns]]
            if market==True:
                df_pivot = df_pivot[df_pivot[df_pivot.columns[1]].isin(hirlis)]

            for metric in metrics:
                matching_cols = [col for col in numeric_cols if col.startswith(metric)]
                if matching_cols:
                    if "Sales" in metric:
                        df_pivot[f"Total {metric}"] = df_pivot[matching_cols].sum(axis=1)
                    else:
                        df_pivot[f"Total {metric}"] = df_pivot[matching_cols].mean(axis=1)
            
            df_pivot = df_pivot.loc[:, ~df_pivot.columns.astype(str).str.contains("nan", case=False, na=False)]
            comp = df_pivot[~df_pivot['Top Brands'].isin(client_brands)]
            value_sales_cols = [col for col in comp.columns if "Value Sales P12M" in col]

            # Get the last (latest) one
            latest_vs_col = value_sales_cols

            # Proceed only if such a column exists
            if latest_vs_col:
                top5_competitors = comp.sort_values(by=latest_vs_col, ascending=False).head(5)
                top5_list = top5_competitors['Top Brands'].tolist()

            else:
                print("No Value Sales column found for sorting.")
            t_filtered = df_pivot[df_pivot['Top Brands'].isin(client_brands) |
                        ((df_pivot['Top Brands'].isin(top5_list)) & (df_pivot['Value Share P12M'] > 0.02))]
            t_filtered['Value Sales P12M'] = t_filtered['Value Sales P12M'].astype(float)
            t_filtered = t_filtered.sort_values('Value Sales P12M', ascending=False)

            
                                
            output[k] = t_filtered

    return output


In [37]:
modified_momentum_retailersfinal=process_dateMomentum(modified_momentum_retailers_new,"Time Period",2,retailers,market=True)
modified_momentum_channelsfinal=process_dateMomentum(modified_momentum_channels_new,"Time Period",2,channels,market=True)
modified_momentum_custfinal=process_dateMomentum(modified_momentum_cust_new,"Time Period",2,cust,market=True)

In [38]:
def add_totals_after_grouping(data_dict):
    updated_dict = {}

    for key, df in data_dict.items():
        if not df.empty:
            first_col = df.columns[0]  # Get the first column name
            df=df[df[first_col]!="Grand Total"]
            numeric_cols = df.select_dtypes(include=['number']).columns  # Get numeric columns

            grouped = df.groupby(first_col)[numeric_cols].sum().reset_index()  # Sum only numeric columns

            new_df_list = []
            for group, group_df in df.groupby(first_col):
                new_df_list.append(group_df)  # Append original group
                
                total_row = grouped[grouped[first_col] == group].copy()  # Get total row
                total_row[first_col] = f"{group} Total"  # Rename the group name for total row
                new_df_list.append(total_row)  # Append total row

            updated_dict[key] = pd.concat(new_df_list, ignore_index=True)  # Combine results

    return updated_dict
variables = [
    "modified_momentum_retailersfinal",
    "modified_momentum_channelsfinal",
    "modified_momentum_custfinal"
]

for var in variables:
    if var in globals():
        globals()[var] = add_totals_after_grouping(globals()[var])


In [39]:
markets_val = [regions_RET + channels_RET+ market_RET  +regions_CHAN +channels_CHAN+ market_CHAN +regions_CUST +channels_CUST+ market_CUST]
markets_val = markets_val[0]
def process_RET_CHAN_Momentum(dataframes, client_brands,areaby,col=" ",lst=[],sec=False):
    # Dictionary to hold all modified DataFrames in a single structure
    modified_data = {}
    for k, df in dataframes.items():
        # Copy and process each DataFrame
        t = df.copy()
        if not t.empty:
            t = t[~t["Top Brands"].fillna("").str.strip().eq("")]
            top_brands_totals = t[(t['Top Brands'].str.contains("Total")) & (t['Top Brands']!="Grand Total") & (~t["Top Brands"].isin([i+" Total" for i in client_brands]))]
            competitors = top_brands_totals.sort_values(by = "Value Sales P12M",ascending=False).head(5)['Top Brands'].to_list()
            competitors = [i.replace(" Total","") for i in competitors]
            t_filtered = t[t['Top Brands'].isin(client_brands) |
                                    ((t['Top Brands'].isin(competitors)) & (t['Value Share P12M'] > 0.02))]
            t_filtered = t_filtered.applymap(lambda x: float(x) if pd.to_numeric(x, errors='coerce') == x else x)
            t_filtered['Value Sales P12M'] = t_filtered['Value Sales P12M'].astype(float)
            t_filtered = t_filtered.sort_values('Value Sales P12M', ascending=False)
            t_filtered = t_filtered[t_filtered.iloc[:, 1].isin(markets_val)]
            if sec and col and col in t_filtered.columns:
                t_filtered = t_filtered[t_filtered[col].isin(lst)]
            filtered_df = t_filtered
            # newKey=k.split(' | ')[0] +" | " +f'{areaby}'
            if not filtered_df.empty:
                if sec==True:
                    filtered_df['Labels'] = filtered_df['Top Brands'] + "/" + filtered_df[filtered_df.columns[1]]+"/" + filtered_df["Sector"]+ "/" + filtered_df[f"{col}"]  
                else :    
                    filtered_df['Labels'] = filtered_df['Top Brands'] + "/" + filtered_df[filtered_df.columns[1]]

                modified_data[k] = pd.concat([modified_data.get(k, pd.DataFrame()), filtered_df], ignore_index=True)
                modified_data[k] = modified_data[k].sort_values('Value Sales P12M', ascending=False)
                modified_data[k] = modified_data[k].drop_duplicates() 
    return modified_data

In [40]:
modified_momentum_retailers = process_RET_CHAN_Momentum(modified_momentum_retailersfinal,client_brands,"Retailers")

modified_momentum_channels = process_RET_CHAN_Momentum(modified_momentum_channelsfinal,client_brands,"Channels")

modified_momentum_cust = process_RET_CHAN_Momentum(modified_momentum_custfinal,client_brands, f"{customareas}")


In [41]:
def merge_dicts(target, *sources):
    for source in sources:
        if source:  # Ensure the source is not None or empty
            target.update(source)
if segments:
    modified_momentum_retailers_seg_new = {}
    merge_dicts(
        modified_momentum_retailers_seg_new, 
        globals().get('modified_momentum_seg_retailer_Region', {}), 
        globals().get('modified_momentum_seg_retailer_Channel', {}), 
        globals().get('modified_momentum_seg_retailer_Market', {})
    )

    modified_momentum_channels_seg_new = {}
    merge_dicts(
        modified_momentum_channels_seg_new, 
        globals().get('modified_momentum_seg_channels_Region', {}), 
        globals().get('modified_momentum_seg_channels_Channel', {}), 
        globals().get('modified_momentum_seg_channels_Market', {})
    )

    modified_momentum_cust_seg_new = {}
    merge_dicts(
        modified_momentum_cust_seg_new, 
        globals().get('modified_momentum_seg_cust_Region', {}), 
        globals().get('modified_momentum_seg_cust_Channel', {}), 
        globals().get('modified_momentum_seg_cust_Market', {})
    )
    
###### SubSegment    
if subsegments:
    modified_momentum_retailers_subseg_new = {}
    merge_dicts(
        modified_momentum_retailers_subseg_new, 
        globals().get('modified_momentum_subseg_retailer_Region', {}), 
        globals().get('modified_momentum_subseg_retailer_Channel', {}), 
        globals().get('modified_momentum_subseg_retailer_Market', {})
    )

    modified_momentum_channels_subseg_new = {}
    merge_dicts(
        modified_momentum_channels_subseg_new, 
        globals().get('modified_momentum_subseg_channels_Region', {}), 
        globals().get('modified_momentum_subseg_channels_Channel', {}), 
        globals().get('modified_momentum_subseg_channels_Market', {})
    )

    modified_momentum_cust_subseg_new = {}
    merge_dicts(
        modified_momentum_cust_subseg_new, 
        globals().get('modified_momentum_subseg_cust_Region', {}), 
        globals().get('modified_momentum_subseg_cust_Channel', {}), 
        globals().get('modified_momentum_subseg_cust_Market', {})
    )    
###### SubCategory   
if subcategories:
    modified_momentum_retailers_subcatg_new= {}
    merge_dicts(
        modified_momentum_retailers_subcatg_new, 
        globals().get('modified_momentum_subcat_retailer_Region', {}), 
        globals().get('modified_momentum_subcat_retailer_Channel', {}), 
        globals().get('modified_momentum_subcat_retailer_Market', {})
    )

    modified_momentum_channels_subcatg_new = {}
    merge_dicts(
        modified_momentum_channels_subcatg_new, 
        globals().get('modified_momentum_subcat_channels_Region', {}), 
        globals().get('modified_momentum_subcat_channels_Channel', {}), 
        globals().get('modified_momentum_subcat_channels_Market', {})
    )

    modified_momentum_cust_subcatg_new = {}
    merge_dicts(
        modified_momentum_cust_subcatg_new, 
        globals().get('modified_momentum_subcat_cust_Region', {}), 
        globals().get('modified_momentum_subcat_cust_Channel', {}), 
        globals().get('modified_momentum_subcat_cust_Market', {})
    )       

In [42]:
if segments:   
    modified_momentum_seg_retailersfinal=process_dateMomentum(modified_momentum_retailers_seg_new,"Time Period",4,retailers,market=True)
    modified_momentum_seg_channelsfinal=process_dateMomentum(modified_momentum_channels_seg_new,"Time Period",4,channels,market=True)
    modified_momentum_seg_custfinal=process_dateMomentum(modified_momentum_cust_seg_new,"Time Period",4,cust,market=True)
if subsegments:   
    modified_momentum_subseg_retailersfinal=process_dateMomentum(modified_momentum_retailers_subseg_new,"Time Period",4,retailers,market=True)
    modified_momentum_subseg_channelsfinal=process_dateMomentum(modified_momentum_channels_subseg_new,"Time Period",4,channels,market=True)
    modified_momentum_subseg_custfinal=process_dateMomentum(modified_momentum_cust_subseg_new,"Time Period",4,cust,market=True)
if subcategories:   
    modified_momentum_subcat_retailersfinal=process_dateMomentum(modified_momentum_retailers_subcatg_new,"Time Period",4,retailers,market=True)
    modified_momentum_subcat_channelsfinal=process_dateMomentum(modified_momentum_channels_subcatg_new,"Time Period",4,channels,market=True)
    modified_momentum_subcat_custfinal=process_dateMomentum(modified_momentum_cust_subcatg_new,"Time Period",4,cust,market=True)        

In [43]:
##### Grouping & Total
variables = []

if segments:
    variables.extend([
        "modified_momentum_seg_retailersfinal",
        "modified_momentum_seg_channelsfinal",
        "modified_momentum_seg_custfinal"
    ])

if subsegments:
    variables.extend([
        "modified_momentum_subseg_retailersfinal",
        "modified_momentum_subseg_channelsfinal",
        "modified_momentum_subseg_custfinal"
    ])

if subcategories:
    variables.extend([
        "modified_momentum_subcat_retailersfinal",
        "modified_momentum_subcat_channelsfinal",
        "modified_momentum_subcat_custfinal"
    ])

for var in variables:
    if var in globals() and globals()[var] is not None:
        globals()[var] = add_totals_after_grouping(globals()[var])


In [44]:
if segments:
    modified_momentum_retailers_seg = process_RET_CHAN_Momentum(modified_momentum_seg_retailersfinal,client_brands,"Retailers","Segment", segments,sec=True)
    modified_momentum_channels_seg = process_RET_CHAN_Momentum(modified_momentum_seg_channelsfinal,client_brands,"Channels","Segment", segments,sec=True)
    modified_momentum_cust_seg = process_RET_CHAN_Momentum(modified_momentum_seg_custfinal,client_brands,f"{customareas}","Segment", segments,sec=True)
    
if subsegments:
    modified_momentum_retailers_subseg = process_RET_CHAN_Momentum(modified_momentum_subseg_retailersfinal,client_brands,"Retailers","SubSegment", subsegments,sec=True)
    modified_momentum_channels_subseg = process_RET_CHAN_Momentum(modified_momentum_subseg_channelsfinal,client_brands,"Channels","SubSegment", subsegments,sec=True)
    modified_momentum_cust_subseg = process_RET_CHAN_Momentum(modified_momentum_subseg_custfinal,client_brands,f"{customareas}","SubSegment", subsegments,sec=True)
    
if subcategories:
    modified_momentum_retailers_subcatg = process_RET_CHAN_Momentum(modified_momentum_subcat_retailersfinal,client_brands,"Retailers","SubCategory", subcategories,sec=True)
    modified_momentum_channels_subcatg = process_RET_CHAN_Momentum(modified_momentum_subcat_channelsfinal,client_brands,"Channels","SubCategory", subcategories,sec=True)
    modified_momentum_cust_subcatg = process_RET_CHAN_Momentum(modified_momentum_subcat_custfinal,client_brands,f"{customareas}","SubCategory", subcategories,sec=True)
        

In [45]:
momentum_dfs_frist=process_dateMomentum(momentum_dfs,"Time Period",2,market=False)

In [46]:
momentum_dfs_new=add_totals_after_grouping(momentum_dfs_frist)

In [47]:
modified_momentum_dfs = {}
for key in momentum_dfs_new.keys():
    t = momentum_dfs_new[key].copy()
    if not t.empty:
        t[f'{BrandOrTopB}'] = t[f'{BrandOrTopB}'].ffill()
        if not t.empty:
            top_brands_totals = t[(t[f'{BrandOrTopB}'].str.contains("Total")) & (t[f'{BrandOrTopB}']!="Grand Total|All Others Total") & (~t[f"{BrandOrTopB}"].isin([i+" Total" for i in client_brands]))]
            competitors = top_brands_totals.sort_values(by = "Value Sales P12M",ascending=False).head(5)[f'{BrandOrTopB}'].to_list()
            competitors = [i.replace(" Total","") for i in competitors]
            t_filtered = t[t[f'{BrandOrTopB}'].isin(client_brands) |
                                    ((t[f'{BrandOrTopB}'].isin(competitors)) & (t['Value Share P12M'] > 0.02))]

            t_filtered = t_filtered.applymap(lambda x: float(x) if pd.to_numeric(x, errors='coerce') == x else x)
            t_filtered['Value Sales P12M'] = t_filtered['Value Sales P12M'].astype(float)
            t_filtered = t_filtered.sort_values('Value Sales P12M', ascending=False)
            t_filtered['Labels'] = t_filtered[f'{BrandOrTopB}'] + "/" + t_filtered['Sector'] 

            if not t_filtered.empty:
                modified_momentum_dfs[key] = t_filtered 

In [48]:
def genrateIndexList(scopeCategory ,chartIndex =0 ,chartCount =4 ):
    if len(scopeCategory) == 0:
        return []
    page = len(scopeCategory) # number of sector or segment or category
 
    lis=[]
    while page >chartCount:
        lis.append(chartCount + chartIndex)
        page -=chartCount
    lis.append(page + chartIndex)
    return [lis]

sectorIndex = []
segmentIndex = []
subsegmentIndex = []
subcategoryIndex = []
retailerREGIndex = []
retailerCHAIndex = []
retailerMARIndex = []
channelREGIndex = []
channelCHAIndex = []
channelMARIndex = []
custREGIndex = []
custCHAIndex = []
custMARIndex = []

if len (sectors)!=0:
    
    for key,value in sharGrowthDf_sec.items():
        sectorIndex+=genrateIndexList(value,4,3)[0]

if len(segments)!=0:
    
    for key,value in sharGrowthDf_seg.items():
        segmentIndex+=genrateIndexList(value,4,3)[0]

if len(subsegments)!=0:
    
    for key,value in sharGrowthDf_subseg.items():
        subsegmentIndex+=genrateIndexList(value,4,3)[0]

if len(subcategories)!=0:
    
    for key,value in sharGrowthDf_subcat.items():
        subcategoryIndex+=genrateIndexList(value,4,3)[0]

if len(regions_RET)!=0:
    for key,value in shareGrowthDf_retailers_REG.items():
        retailerREGIndex+=genrateIndexList(value,4,3)[0]

if len(channels_RET)!=0:
    for key,value in shareGrowthDf_retailers_CHAN.items():
        retailerCHAIndex+=genrateIndexList(value,4,3)[0]
if len(market_RET)!=0:
    for key,value in shareGrowthDf_retailers_MAR.items():
        retailerMARIndex+=genrateIndexList(value,4,3)[0]
        
if len(regions_CHAN)!=0:
    for key,value in shareGrowthDf_channels_REG.items():
        channelREGIndex+=genrateIndexList(value,4,3)[0]
if len(channels_CHAN)!=0:
    for key,value in shareGrowthDf_channels_CHAN.items():
        channelCHAIndex+=genrateIndexList(value,4,3)[0]
if len(market_CHAN)!=0:
    for key,value in shareGrowthDf_channels_MAR.items():
        channelMARIndex+=genrateIndexList(value,4,3)[0]

if len(regions_CUST)!=0:
    for key,value in shareGrowthDf_custom_REG.items():
        custREGIndex+=genrateIndexList(value,4,3)[0]
if len(channels_CUST)!=0:
    for key,value in shareGrowthDf_custom_CHAN.items():
        custCHAIndex+=genrateIndexList(value,4,3)[0]
if len(market_CUST)!=0:
    for key,value in shareGrowthDf_custom_MAR.items():
        custMARIndex+=genrateIndexList(value,4,3)[0]


# Slide Creations

### First Landscape Deck

In [49]:
index = [
    *[0]*(2+(1 if sectors else 0)+(1 if segments else 0)+(1 if subcategories else 0)+(1 if subsegments else 0)),
           *[1]*(2+(1 if sectors else 0)+(1 if segments else 0)+(1 if subcategories else 0)+(1 if subsegments else 0)),
              #  *[2]*((1 if sectors else 0) +(1 if segments else 0)+(1 if subcategories else 0)+(1 if subsegments else 0)),
              #  *[2]*section_number,
               *[3]*((1 if sectors else 0) +(1 if segments else 0)+(1 if subcategories else 0)+(1 if subsegments else 0)),
             *[3]*section_number_Avg,
                ]
index=[i for i in index if i !=[]]

duplication_1 = [len(modified_manuf_dfs_new.keys()), len(modified_brands_share_new.keys()),len(modified_sectors_dfs_new.keys())if sectors else 0,len(modified_segment_dfs_new.keys())if segments else 0,len(modified_subsegment_dfs_new.keys())if subsegments else 0,len(modified_subcategories_dfs_new.keys())if subcategories else 0]
duplication_2 = [len(modified_manuf_dfs_new.keys()), len(modified_brands_share_new.keys()), len(modified_sectors_dfs_new.keys())if sectors else 0, len(modified_segment_dfs_new.keys())if segments else 0,len(modified_subsegment_dfs_new.keys())if subsegments else 0,len(modified_subcategories_dfs_new.keys())if subcategories else 0]
# duplication_3 = [len(modified_sectors_P12M_new.keys()) if sectors else 0, len(modified_segment_P12M_new.keys())if segments else 0, len(modified_subsegment_P12M_new.keys())if subsegments else 0, len(modified_subcategories_P12M_new.keys())if subcategories else 0,*duplication_num]
duplication_4 = [len(modified_sectors_clients_new.keys())if sectors else 0,len(modified_segment_clients_new.keys())if segments else 0, len(modified_subsegment_clients_new.keys())if subsegments else 0, len(modified_subcategories_clients_new.keys())if subcategories else 0, *duplication_num_Avg]

duplication = duplication_1 + duplication_2  + duplication_4 #+ duplication_3  
duplication = [item for item in duplication if item !=0]

section_names_slide1 = ["Market Trends by Manufacturer","Market Trends by Brands","Market Trends by Sectors"] + (["Market Trends by Segments"] if len(segments)>0 else [])+(["Market Trends by SubSegments"] if len(subsegments)>0 else [])+(["Market Trends by SubCategory"] if len(subcategories)>0 else [])
section_names_slide2 = ["Market Concentration By Manufacturer", "Market Concentration By Brands", "Market Concentration By Sectors"]+ (["Market Concentration By Segments"] if len(segments)>0 else [])+(["Market Concentration by SubSegments"] if len(subsegments)>0 else [])+(["Market Concentration by SubCategory"] if len(subcategories)>0 else [])
# section_names_slide3 = (["Market Growth By Sectors"]if len(sectors)>0 else [])+(["Market Growth By Segments"]if len(segments)>0 else [])+(["Market Growth By SubSegments"]if len(subsegments)>0 else [])+(["Market Growth By SubCategory"]if len(subcategories)>0 else [])+[*section_name_Growth]
section_names_slide4 = (["Value Vs AvgPrice By Sectors"]if len(sectors)>0 else [])+(["Value Vs AvgPrice By Segments"]if len(segments)>0 else [])+(["Value Vs AvgPrice By SubSegments"]if len(subsegments)>0 else [])+(["Value Vs AvgPrice By SubCategory"]if len(subcategories)>0 else [])+[*section_name_Avg]


section_names = [ *section_names_slide1 ,*section_names_slide2, *section_names_slide4                   
                 ]#*section_names_slide3,
path = os.getcwd() + '\Landscape base.pptx'
new_pre = os.getcwd() + '\Landscape duplicate Market(Trends).pptx'



In [50]:
print(len(index))
print(index)
print(len(duplication))
print(duplication)
print(len(section_names))
# print(section_names)

15
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3]
15
[120, 120, 6, 6, 6, 120, 120, 6, 6, 6, 72, 72, 72, 68, 68]
15


In [51]:
print(sum(duplication))

868


In [66]:
slideDuplication(index,duplication,section_names,path,new_pre)

In [52]:
prs = Presentation(new_pre)

In [53]:
def calculate_position(end):
    return sum(duplication[i] * (1 if isinstance(index[i], int) else len(index[i])) for i in range(end))

#### Slide 1_ Market_Trends

In [54]:
p=0
Market_Trends(prs, list(modified_manuf_dfs_new.keys()), modified_manuf_dfs_new, modified_manuf_totals_new, client_manuf ,position = calculate_position(p), slide_by = f'{ManufOrTopC}', years=years)
p+=1

Market_Trends(prs, list(modified_brands_share_new.keys()), modified_brands_share_new, modified_brands_totals_new, client_brands ,position =calculate_position(p), slide_by = f'{BrandOrTopB}', years=years)
p+=1

if len(sectors)!=0:
    Market_Trends(prs, list(modified_sectors_dfs_new.keys()), modified_sectors_dfs_new, sectors_totals_new, sectors ,position = calculate_position(p), slide_by = 'Sector', years=years)
    p+=1

if len(segments)!=0:
    Market_Trends(prs, list(modified_segment_dfs_new.keys()), modified_segment_dfs_new, segment_totals_new, segments ,position = calculate_position(p), slide_by = 'Segment', years=years)
    p+=1

    
if len(subsegments)!=0:
    Market_Trends(prs, list(modified_subsegment_dfs_new.keys()), modified_subsegment_dfs_new, subsegment_totals_new, subsegments ,position = calculate_position(p), slide_by = 'SubSegment', years=years)
    p+=1

if len(subcategories)!=0:
    Market_Trends(prs, list(modified_subcategories_dfs_new.keys()), modified_subcategories_dfs_new, subcategories_totals_new, subcategories ,position = calculate_position(p), slide_by = 'SubCategory', years=years)
    p+=1


In [55]:
for k in modified_brands_share_new.keys():
    modified_brands_share_new[k][f'{BrandOrTopB}'] = modified_brands_share_new[k][f'{BrandOrTopB}'].str.replace("Private Label", "PL")

#### Slide 2_Market_Concentration

In [56]:
Market_Concentration(prs,list(modified_manuf_dfs_new.keys()), modified_manuf_dfs_new, modified_manuf_totals_new, client_manuf ,position =  calculate_position(p), slide_by = f'{ManufOrTopC}',years=years)
p+=1
Market_Concentration(prs,list(modified_brands_share_new.keys()), modified_brands_share_new, modified_brands_totals_new, client_brands ,position =  calculate_position(p), slide_by = f'{BrandOrTopB}',years=years)
p+=1
if len(sectors)!=0:
    Market_Concentration(prs,list(modified_sectors_dfs_new.keys()), modified_sectors_dfs_new, sectors_totals_new, sectors ,position =  calculate_position(p), slide_by = 'Sector',years=years)
    p+=1
if len(segments)!=0:
    Market_Concentration(prs,list(modified_segment_dfs_new.keys()), modified_segment_dfs_new, segment_totals_new, segments ,position =  calculate_position(p), slide_by = 'Segment',years=years)
    p+=1

if len(subsegments)!=0:
    Market_Concentration(prs,list(modified_subsegment_dfs_new.keys()), modified_subsegment_dfs_new, subsegment_totals_new, subsegments ,position =  calculate_position(p), slide_by = 'SubSegment',years=years)
    p+=1

if len(subcategories)!=0:
    Market_Concentration(prs,list(modified_subcategories_dfs_new.keys()), modified_subcategories_dfs_new, subcategories_totals_new, subcategories ,position =  calculate_position(p), slide_by = 'SubCategory',years=years)
    p+=1


#### Slide 3_Market_Growth_Contribution

###### By Categpry/Sec/Seg/SubSeg/SubCat

In [57]:
# def Market_Growth(prs, list_duplicates, modified_df_P12M, dfs_dya,position = 0, slide_by = '',slide_for = ''):
#     """
#         Generates market growth analysis slides with charts and tables.

#         Parameters:
#             prs (Presentation): PowerPoint presentation object.
#             list_duplicates (list): List of duplicate names for identifying slides.
#             modified_df_P12M (dict): Dictionary of modified DataFrames for each duplicate name.
#             dfs_dya (dict): Dictionary of DYA (Year-Ago) DataFrames for each duplicate name.
#             position (int, optional): Position offset for slides. Defaults to 0.
#             slide_by (str, optional): Slide grouping criteria. Defaults to ''.
#             slide_for (str, optional): Specific slide criteria. Defaults to ''.
#     """  

#     for slidenum in range(len(list_duplicates)):
#         shapes = prs.slides[slidenum + position].shapes
#         charts = []
#         tables = []
#         title = shapes.title.text

#         titleNumber = get_shape_number(shapes, "Market growth contributors | By Sectors | Confectionary | National | P12M | Index vs YA Value Sales DYA: 3") #9
#         headerNumber = get_shape_number(shapes, "Market growth contributors (Replace with So What)") #10
#         datasourcenum = get_shape_number(shapes, "  DATA SOURCE: Trade Panel/Retailer Data | Ending Dec 2022 ") #8

#         if titleNumber is not None:
#             shapes[datasourcenum].text = data_source

#             dya_value = int(dfs_dya[list_duplicates[slidenum]]['Value Sales IYA'] *100 -100)
#             if slide_for:
#                 shapes[titleNumber].text = f'Market growth contributors | By {slide_for} | ' + list_duplicates[slidenum] + ' | P12M | Index vs YA Value Sales | DYA: ' + ('+' if dya_value > 0 else '')+ str(dya_value)+ "%".replace("  "," ")
#             else:
#                 shapes[titleNumber].text = f'Market growth contributors | By {slide_by} | ' + list_duplicates[slidenum] + ' | P12M | Index vs YA Value Sales | DYA: ' +('+' if dya_value > 0 else '')+ str(dya_value)+ "%".replace("  "," ")
                

#         for shape in shapes:
#             if shape.has_chart:
#                 shape_id = shape.shape_id
#                 charts.append(shape)
#             if shape.has_table:
#                 shape_id = shape.shape_id
#                 tables.append(shape)

#         cols = {
#             'C2': 'Growth Contribution',
#             'C1': 'WoB %'
#         }

#         for chartnum in range(len(charts)):
#             chart = charts[chartnum].chart
#             chart_name = charts[chartnum].name
#             chart_type = chart.chart_type
#             chart_data = CategoryChartData()

#             chart_data.categories = modified_df_P12M[list_duplicates[slidenum]][f'{slide_by}'].astype(str).tolist()
#             modified_df_P12M[list_duplicates[slidenum]] = modified_df_P12M[list_duplicates[slidenum]].replace(np.nan,0)
#             if chart_name in cols.keys():
#                 chart_data.add_series(cols[chart_name], modified_df_P12M[list_duplicates[slidenum]][cols[chart_name]])
#                 chart.replace_data(chart_data)

#         table = tables[1].table
        
        
#         num_columns_to_remove = (len(table.columns) - modified_df_P12M[list_duplicates[slidenum]].shape[0]) - 1  # Specify the number of rows to remove from the end
#         for _ in range(num_columns_to_remove):
#             col = table.columns[modified_df_P12M[list_duplicates[slidenum]].shape[0]]
#             remove_col(table, col)

#         table_width = Inches(9.22)  # Specify the desired table height
#         # table.columns[0].width=524500
#         table.columns[0].width=Inches(0.86)

#         total_col_width = table_width - table.columns[0].width
#         num_columns = len(table.columns)-1  # Exclude the first row

#         if num_columns > 0:
#             cell_width = total_col_width / num_columns
#             for col in range(1, table.columns.__len__()):
#                 table.columns[col].width = int(cell_width)
                        
#         data_ = ['Value Sales', 'Value Sales IYA']

#         for i, row in enumerate(table.rows):
#             for j, cell in enumerate(row.cells):
#                 cell.fill.solid()
#                 if j == 0:
#                     if i == 0:
#                         cell.text = cell.text + " " + currency
#                         cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 255, 255)
#                         cell.text_frame.paragraphs[0].font.bold = False
#                         cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
#                         cell.text_frame.paragraphs[0].font.size = Pt(7)
#                         cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
#                         # "Value Sales\n(‘000’000)"
#                 if j != 0:
#                     if i == 0:
#                         # if sign == "Before":
#                         cell.text = str(round(modified_df_P12M[list_duplicates[slidenum]][[data_[i]]].iloc[j - 1].values[0] / 1000000, 1))
#                         # else:
#                         #     cell.text = str(round(modified_df_P12M[list_duplicates[slidenum]][[data_[i]]].iloc[j - 1].values[0] / 1000000, 1)) +currency
                          
#                         cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
#                         cell.text_frame.paragraphs[0].font.bold = False
#                         cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
#                         cell.text_frame.paragraphs[0].font.size = Pt(7)
#                         cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        

#                     else:
#                         cell.text = str(round(modified_df_P12M[list_duplicates[slidenum]][[data_[i]]].iloc[j - 1].values[0] * 100))
#                         if int(cell.text) <= 90:
#                             cell.text_frame.paragraphs[0].font.size = Pt(7)
#                             cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
#                             cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(192, 0, 0)
#                             cell.text_frame.paragraphs[0].font.name = 'Nexa Book'

#                         elif int(cell.text) <= 98:
#                             cell.text_frame.paragraphs[0].font.size = Pt(7)
#                             cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
#                             cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 128, 128)
#                             cell.text_frame.paragraphs[0].font.name = 'Nexa Book'

#                         elif int(cell.text) >= 110:
#                             cell.text_frame.paragraphs[0].font.size = Pt(7)
#                             cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
#                             cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(0, 160, 151)
#                             cell.text_frame.paragraphs[0].font.name = 'Nexa Book'

#                         elif int(cell.text) >= 102:
#                             cell.text_frame.paragraphs[0].font.size = Pt(7)
#                             cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
#                             cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(178, 223, 220)
#                             cell.text_frame.paragraphs[0].font.name = 'Nexa Book'

#                         else:
#                             cell.text_frame.paragraphs[0].font.size = Pt(7)
#                             cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
#                             cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
#                             cell.text_frame.paragraphs[0].font.name = 'Nexa Book'

#                 else:
#                     cell.text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
#                     cell.text_frame.paragraphs[0].font.size = Pt(7)
#                     cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER


In [58]:
# Market_Growth(prs,list(modified_sectors_P12M_new.keys()), modified_sectors_P12M_new, dya_sectors_new, position = calculate_position(p) , slide_by = 'Sector')
# p+=1
# if len(segments) !=0:
#     Market_Growth(prs,list(modified_segment_P12M_new.keys()), modified_segment_P12M_new, dya_segment_new, position = calculate_position(p) , slide_by = 'Segment') 
#     p+=1

# if len(subsegments) !=0:
#     Market_Growth(prs,list(modified_subsegment_P12M_new.keys()), modified_subsegment_P12M_new, dya_subsegment_new, position = calculate_position(p) , slide_by = 'SubSegment') 
#     p+=1

# if len(subcategories) !=0:
#     Market_Growth(prs,list(modified_subcategories_P12M_new.keys()), modified_subcategories_P12M_new, dya_subcategories_new, position = calculate_position(p) , slide_by = 'SubCategory') 
#     p+=1


###### By Region/Channel/Market

In [74]:
# for key,val in market_list_Growth.items():
#     Market_Growth(prs, list(val[1].keys()), val[1], val[2], position = calculate_position(p), slide_by = val[3], slide_for = val[4])
#     p+=1


#### Slide4_ValueSales_AvgPrice

In [59]:
market_list=["NATIONAL"] + regions_RET + channels_RET+market_RET +  regions_CHAN +channels_CHAN+ market_CHAN+regions_CUST +channels_CUST+ market_CUST
manuf_brand_list = client_manuf + client_brands

if len(sectors):
    ValueSales_AvgPrice(prs, list(modified_sectors_clients_new.keys()), modified_sectors_P12M_new, modified_sectors_clients_new,manuf_brand_list, market_list,position = calculate_position(p), slide_by = 'Sector')
    ValueSales_AvgPrice(prs, list(modified_sectors_clients_new.keys()), modified_sectors_P12M_new, modified_sectors_clients_new,manuf_brand_list, market_list,position = calculate_position(p), slide_by = 'Sector')
    p+=1

if len(segments)!=0:
    ValueSales_AvgPrice(prs, list(modified_segment_clients_new.keys()), modified_segment_P12M_new, modified_segment_clients_new,manuf_brand_list,market_list, position = calculate_position(p) , slide_by = 'Segment')
    ValueSales_AvgPrice(prs, list(modified_segment_clients_new.keys()), modified_segment_P12M_new, modified_segment_clients_new,manuf_brand_list,market_list, position = calculate_position(p) , slide_by = 'Segment')
    p+=1

if len(subsegments)!=0:
    ValueSales_AvgPrice(prs, list(modified_subsegment_clients_new.keys()), modified_subsegment_P12M_new, modified_subsegment_clients_new,manuf_brand_list,market_list, position = calculate_position(p) , slide_by = 'SubSegment')
    ValueSales_AvgPrice(prs, list(modified_subsegment_clients_new.keys()), modified_subsegment_P12M_new, modified_subsegment_clients_new,manuf_brand_list,market_list, position = calculate_position(p) , slide_by = 'SubSegment')
    p+=1

if len(subcategories)!=0:
    ValueSales_AvgPrice(prs, list(modified_subcategories_clients_new.keys()), modified_subcategories_P12M_new, modified_subcategories_clients_new,manuf_brand_list,market_list, position = calculate_position(p) , slide_by = 'SubCategory')
    ValueSales_AvgPrice(prs, list(modified_subcategories_clients_new.keys()), modified_subcategories_P12M_new, modified_subcategories_clients_new,manuf_brand_list,market_list, position = calculate_position(p) , slide_by = 'SubCategory')
    p+=1

catg_list=[f'{categories[0]}'] + sectors + segments + subsegments + subcategories
for key,val in market_list_Avg.items():
    ValueSales_AvgPrice(prs, list(val[2].keys()), val[1], val[2],manuf_brand_list,catg_list ,position = calculate_position(p), slide_by = val[3], slide_for = val[4])
    ValueSales_AvgPrice(prs, list(val[2].keys()), val[1], val[2],manuf_brand_list,catg_list ,position = calculate_position(p), slide_by = val[3], slide_for = val[4])
    p+=1

In [60]:
for k in modified_brands_P12M_new.keys():
    modified_brands_P12M_new[k] = modified_brands_P12M_new[k].rename(columns = { "Value Share P12M":"Value Share", "Share DYA P12M":"Share DYA", "Av Price/KG P12M":"Av Price/KG"})
    modified_brands_P12M_new[k] = modified_brands_P12M_new[k].sort_values('Value Share', ascending = False)
    modified_brands_P12M_new[k] = modified_brands_P12M_new[k][modified_brands_P12M_new[k]['Value Share']!=0]    
    modified_brands_P12M_new[k]['Sorting'] = np.where(modified_brands_P12M_new[k][f'{BrandOrTopB}'] == 'Others', 1, 0)
    modified_brands_P12M_new[k] = modified_brands_P12M_new[k].sort_values(['Sorting','Value Share'], ascending = [True, False]).drop(columns = 'Sorting')
    
for k in modified_manuf_P12M_new.keys():
    modified_manuf_P12M_new[k] = modified_manuf_P12M_new[k].sort_values('Total Value Share', ascending=False)
    modified_manuf_P12M_new[k]['Sorting'] = np.where(modified_manuf_P12M_new[k][f'{ManufOrTopC}'] == 'Others', 1, 0)
    modified_manuf_P12M_new[k] = modified_manuf_P12M_new[k].sort_values(['Sorting', "Total Value Share"], ascending = [True, False]).drop(columns = 'Sorting')
    modified_manuf_P12M_new[k] = modified_manuf_P12M_new[k][modified_manuf_P12M_new[k]['Value Share P12M']!=0]

In [61]:
outputPath = os.getcwd()+f"\\Final Output\\Landscape Market(trend-Concent-growth)&ValAvgPrice_{datetime.today().strftime("%d-%m")}.pptx"
prs.save(outputPath)

## Second Landscape Deck: Category Trend - Overview, Share Evolution

In [62]:
index = [8, 9,9,10]
index=[i for i in index if i !=[]]


duplication_7 = [len(modified_calendar_new.keys())]
duplication_8_bymanuf = [len(modified_manuf_evolution_sorted_new.keys())]
duplication_8 = [len(modified_brands_evolution_sorted_new.keys())]
duplication_9 = [len(categories_overview_dfs.keys())]

duplication = duplication_7 + duplication_8_bymanuf+duplication_8 + duplication_9
duplication = [item for item in duplication if item !=0]


section_names_slide7 =  ["Category Trends"]
section_names_slide8_bymanuf =  ['Share Evolution By Manuf']
section_names_slide8 =  ['Share Evolution By Brand']
section_names_slide9 =  ["Category Overview"]

section_names = [*section_names_slide7,*section_names_slide8_bymanuf,*section_names_slide8,*section_names_slide9]

path = os.getcwd() + '\Landscape base.pptx'
new_pre = os.getcwd() + '\Landscape duplicate Category(Trend-Overview).pptx'



In [63]:
print(len(index))
print(index)
print(len(duplication))
print(duplication)
print(len(section_names))
print(section_names)

print(sum(duplication))


4
[8, 9, 9, 10]
4
[186, 120, 120, 6]
4
['Category Trends', 'Share Evolution By Manuf', 'Share Evolution By Brand', 'Category Overview']
432


In [64]:
# slideDuplication(index,duplication,section_names,path,new_pre)

In [65]:
prs = Presentation(new_pre)

def calculate_position(end):
    return sum(duplication[i] * (1 if isinstance(index[i], int) else len(index[i])) for i in range(end))

#### Slide9_Category_Trend

In [66]:
p = 0
Category_Trend(prs,list(modified_calendar_new.keys()), modified_calendar_new, position = calculate_position(p))
p+=1

#### Slide10_Share_Evolution

In [67]:
for k in modified_brands_evolution_sorted_new.keys():
    t = modified_brands_evolution_sorted_new[k].copy()
    t['Year'] = t['Year'].astype(str)
    mask = t['Year'].str.contains('Total', na=False)
    t.loc[mask, f'{BrandOrTopB}'] = 'Total'
    t.loc[mask, 'Year'] = t.loc[mask, 'Year'].str.replace(' Total', '', regex=True)
    t['Tot Sort'] = np.where(t[f'{BrandOrTopB}'] == 'Total', 0, 1)
    t = t.sort_values(['Year', 'Tot Sort'], ascending=[True, True])
    t = t.drop(columns='Tot Sort')
    modified_brands_evolution_sorted_new[k] = t

for k in modified_manuf_evolution_sorted_new.keys():
    t = modified_manuf_evolution_sorted_new[k].copy()
    t['Year'] = t['Year'].astype(str)  
    mask = t['Year'].str.contains('Total', na=False)
    t.loc[mask, f'{ManufOrTopC}'] = 'Total'
    t.loc[mask, 'Year'] = t.loc[mask, 'Year'].str.replace(' Total', '', regex=True)
    t['Tot Sort'] = np.where(t[f'{ManufOrTopC}'] == 'Total', 0, 1)
    t = t.sort_values(['Year', 'Tot Sort'], ascending=[True, True])
    t = t.drop(columns='Tot Sort')
    modified_manuf_evolution_sorted_new[k] = t

In [68]:
Share_Evolution(prs, list(modified_manuf_evolution_sorted_new.keys()), modified_manuf_evolution_sorted_new, position = calculate_position(p),colname=f"{ManufOrTopC}")
p+=1
Share_Evolution(prs, list(modified_brands_evolution_sorted_new.keys()), modified_brands_evolution_sorted_new, position = calculate_position(p),colname=f'{BrandOrTopB}')
p+=1

### Slide11_CategoryOverview

In [69]:
CategoryOverview(prs,modified_categories_dfs_new,col='MonthYear',position=calculate_position(p),scope = sectors, slide_by = 'Sector')
p+=1

In [70]:
outputPath = os.getcwd()+f"\\Final Output\\Landscape Category(Trend-Overview)_{datetime.today().strftime("%d-%m")}.pptx"
prs.save(outputPath)

# Third Landscape Deck (Share and Growth)

In [71]:
markets_name = ['regions_RET' , 'channels_RET','market_RET'  ,'regions_CHAN' ,'channels_CHAN', 'market_CHAN' ,'regions_CUST' ,'channels_CUST', 'market_CUST']
markets_val = [regions_RET , channels_RET,market_RET  ,regions_CHAN ,channels_CHAN, market_CHAN ,regions_CUST ,channels_CUST, market_CUST]
slide_by = ['Region','Channel','Market']*3
slide_for = [*['Retailer']*3,*['Channel']*3,*[customareas]*3]
market_list_Avg={}
duplication_num_Avg = []
section_names_slide_13 = []
categorizations = [
    ("Category", categories),
    ("Sector", sectors),
    ("Segment", segments),
    ("SubSegment", subsegments),
    ("SubCategory", subcategories),

]
for cat_name, cat_list in categorizations:
    if cat_list:
        for idx,val in enumerate(markets_val):
            if len(val)>0:
                market_list_Avg[markets_name[idx]]=[val,markets_dfs[idx],client_dfs[idx],slide_by[idx],slide_for[idx]]
                section_names_slide_13.append(f"Share and Growth By {cat_name} for {slide_by[idx]} For {slide_for[idx]}")


In [72]:
retailerDup = [len(retailerCatDic),len(retailerSecDic),len(retailerSegDic),len(retailerSubSegDic),len(retailerSubCatDic)]*retMarketLength
channelDup = [len(channelCatDic),len(channelSecDic),len(channelSegDic),len(channelSubSegDic),len(channelSubCatDic)]*chanMarketLength
custDup = [len(custCatDic),len(custSecDic),len(custSegDic),len(custSubSegDic),len(custSubCatDic)]*custMarketLength


In [73]:
slide13Index=[*[12]* (((5 -((not sectors) + (not segments) + (not subsegments)+ (not subcategories)) )) *(9 - ((not regions_RET)+(not channels_RET) + (not market_RET)+ (not regions_CHAN)+ (not channels_CHAN)+ (not market_CHAN)+ (not regions_CUST)+ (not channels_CUST)+ (not market_CUST))  ))]

slide13Dup = retailerDup + channelDup + custDup
slide_12_13_index = [*[11]*(4 -((not sectors) + (not segments) + (not subsegments)+ (not subcategories)) ) ]+slide13Index

duplication = [len(cea_sectors_clients_new.keys()),len(cea_segment_clients_new.keys()),len(cea_subsegment_clients_new.keys()),len(cea_subcategory_clients_new.keys())] + slide13Dup
duplication_12_13 =[ i for i in duplication if i !=0 ]
section_names_slide11 =  (["Sales and Growth By Sector"] if sectors else [])+(["Sales and Growth By Segment"] if len(segments)>0 else [])+(["Sales and Growth By SubSegment"] if len(subsegments)>0 else [])+(["Sales and Growth By SubCategories"] if len(subcategories)>0 else [])



section_names_12_13 = [*section_names_slide11,*section_names_slide_13]

In [74]:
momentum_index=[*[13]* (1+((4 -((not segments) + (not subsegments)+ (not subcategories)) )) *(3 - ((not retailers)+(not channels) + (not cust))  ))]

momentum_index=[i for i in momentum_index if i !=[]]

momentum_dupl_1 = [(len(modified_momentum_retailers.keys())if retailers else 0), (len(modified_momentum_channels.keys())if channels else 0), (len(modified_momentum_cust.keys())if cust else 0)]
momentum_dupl_2 = [(len(modified_momentum_retailers_seg.keys())if retailers else 0), (len(modified_momentum_channels_seg.keys())if channels else 0), (len(modified_momentum_cust_seg.keys())if cust else 0)]if segments else [0]
momentum_dupl_3 = [(len(modified_momentum_retailers_subseg.keys())if retailers else 0), (len(modified_momentum_channels_subseg.keys())if channels else 0), (len(modified_momentum_cust_subseg.keys())if cust else 0)]if subsegments else [0]
momentum_dupl_4 = [(len(modified_momentum_retailers_subcatg.keys())if retailers else 0), (len(modified_momentum_channels_subcatg.keys())if channels else 0), (len(modified_momentum_cust_subcatg.keys())if cust else 0)]if subcategories else [0]
momentum_dupl_5 = [len(modified_momentum_dfs.keys())]


momentum_duplication = momentum_dupl_1 + momentum_dupl_2 + momentum_dupl_3 + momentum_dupl_4 + momentum_dupl_5 
momentum_duplication = [item for item in momentum_duplication if item !=0]

momentum_section_name1 = (['Momentum Analysis By Retailers'] if len(retailers)>0 else [] ) + ([ 'Momentum Analysis By Channels'] if len(channels)>0 else [] ) + ([ f'Momentum Analysis By {customareas}'] if len(cust)>0 else [])
momentum_section_name2 = ((['Momentum Analysis By Retailers By Segments'] if len(retailers)>0 else [] ) + ([ 'Momentum Analysis By Channels By Segments'] if len(channels)>0 else [] ) + ([ f'Momentum Analysis By {customareas} By Segments'] if len(cust)>0 else [] ))if segments else []
momentum_section_name3 = ((['Momentum Analysis By Retailers By SubSegments'] if len(retailers)>0 else [] ) + ([ 'Momentum Analysis By Channels By SubSegments'] if len(channels)>0 else [] ) + ([ f'Momentum Analysis By {customareas} By SubSegments'] if len(cust)>0 else [] ))if subsegments else []
momentum_section_name4 = ((['Momentum Analysis By Retailers By SubCategories'] if len(retailers)>0 else [] ) + ([ 'Momentum Analysis By Channels By SubCategories'] if len(channels)>0 else [] ) + ([ f'Momentum Analysis By {customareas} By SubCategories'] if len(cust)>0 else [] ))if subcategories else []
momentum_section_name5 = "Momentum Analysis By Sectors"

momentum_section_names = [ *momentum_section_name1,*momentum_section_name2,
                  *momentum_section_name3, *momentum_section_name4, momentum_section_name5  
                 ]


In [75]:
index=[
     4,
                retailerREGIndex if regions_RET else [], retailerREGIndex if regions_RET else [], 
                retailerCHAIndex if channels_RET else [], retailerCHAIndex if channels_RET else [], 
                retailerMARIndex if market_RET else [], retailerMARIndex if market_RET else [], 
                channelREGIndex if regions_CHAN else [], channelREGIndex if regions_CHAN else [], 
                channelCHAIndex if channels_CHAN else [], channelCHAIndex if channels_CHAN else [], 
                channelMARIndex if market_CHAN else [], channelMARIndex if market_CHAN else [], 
                custREGIndex if regions_CUST else [], custREGIndex if regions_CUST else [], 
                custCHAIndex if channels_CUST else [], custCHAIndex if channels_CUST else [],  
                custMARIndex if market_CUST else [], custMARIndex if market_CUST else [],           
                sectorIndex if sectors else [],sectorIndex if sectors else [],
                segmentIndex if segments else [],segmentIndex if segments else [],
                subsegmentIndex if subsegments else [],subsegmentIndex if subsegments else [],
                subcategoryIndex if subcategories else [],subcategoryIndex if subcategories else []] + slide_12_13_index + momentum_index
index=[i for i in index if i !=[]]

duplication_5 = [len(modified_manuf_P12M_new.keys())] 
duplication_6area = [1 if regions_RET else 0, 1 if regions_RET else 0,1 if channels_RET else 0, 1 if channels_RET else 0,1 if market_RET else 0, 1 if market_RET else 0,1 if regions_CHAN else 0, 1 if regions_CHAN else 0,1 if channels_CHAN else 0, 1 if channels_CHAN else 0,1 if market_CHAN else 0, 1 if market_CHAN else 0,1 if regions_CUST else 0, 1 if regions_CUST else 0, 1 if channels_CUST else 0, 1 if channels_CUST else 0, 1 if market_CUST else 0, 1 if market_CUST else 0]
duplication_6 = [1 if sectors else 0, 1 if sectors else 0, 1 if segments else 0, 1 if segments else 0, 1 if subsegments else 0, 1 if subsegments else 0, 1 if subcategories else 0, 1 if subcategories else 0]

duplication = duplication_5 + duplication_6area + duplication_6 + duplication_12_13 + momentum_duplication

duplication = [item for item in duplication if item !=0]

section_names_slide5 = ['Share and Growth By Manufacturer/Brands']
section_names_slide6area = (['Share And Growth By Manufacturer By Retailer', 'Share And Growth By Brands By Retailer'] if len(regions_RET)>0 else [] ) +(['Share And Growth By Manufacturer By Retailer', 'Share And Growth By Brands By Retailer'] if len(channels_RET)>0 else [] ) +(['Share And Growth By Manufacturer By Retailer', 'Share And Growth By Brands By Retailer'] if len(market_RET)>0 else [] ) + (['Share And Growth By Manufacturer By Channel', 'Share And Growth By Brands By Channel'] if len(regions_CHAN)>0 else [] ) + (['Share And Growth By Manufacturer By Channel', 'Share And Growth By Brands By Channel'] if len(channels_CHAN)>0 else [] ) + (['Share And Growth By Manufacturer By Channel', 'Share And Growth By Brands By Channel'] if len(market_CHAN)>0 else [] ) + (['Share And Growth By Manufacturer By '+ customareas, 'Share And Growth By Brands By '+ customareas] if len(regions_CUST)>0 else [] ) + (['Share And Growth By Manufacturer By '+ customareas, 'Share And Growth By Brands By '+ customareas] if len(channels_CUST)>0 else [] ) + (['Share And Growth By Manufacturer By '+ customareas, 'Share And Growth By Brands By '+ customareas] if len(market_CUST)>0 else [] )
section_names_slide6 = (['Share And Growth By Manufacturer By Sector', 'Share And Growth By Brands By Sector'] if len(sectors)>0 else [] ) + (['Share And Growth By Manufacturer By Segment', 'Share And Growth By Brands By Segment'] if len(segments)>0 else [] ) + (['Share And Growth By Manufacturer By SubSegment', 'Share And Growth By Brands By SubSegment'] if len(subsegments)>0 else [] )  + (['Share And Growth By Manufacturer By SubCategory', 'Share And Growth By Brands By SubCategory'] if len(subcategories)>0 else [] )


section_names=[*section_names_slide5,*section_names_slide6area,*section_names_slide6] + section_names_12_13 + momentum_section_names
path = os.getcwd() + '\Landscape base.pptx'
new_pre = os.getcwd() + '\Landscape duplicate ShareGrowth.pptx'


In [94]:
momentum_duplication

[20, 20, 1, 1, 1, 1, 6]

In [95]:
print(len(index))
print(index)
print(len(duplication))
print(duplication)
print(len(section_names))
print(section_names)

29
[4, [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6], [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6], [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7], [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7], [7, 5, 7, 5, 7, 5, 7, 5, 7, 5, 7, 5], [7, 5, 7, 5, 7, 5, 7, 5, 7, 5, 7, 5], [7, 7, 7, 5, 7, 7, 7, 5, 7, 7, 7, 5, 7, 7, 7, 5, 7, 7, 7, 5, 7, 7, 7, 5], [7, 7, 7, 5, 7, 7, 7, 5, 7, 7, 7, 5, 7, 7, 7, 5, 7, 7, 7, 5, 7, 7, 7, 5], [7, 7, 5, 7, 7, 5, 7, 7, 5, 7, 7, 5, 7, 7, 5, 7, 7, 5], [7, 7, 5, 7, 7, 5, 7, 7, 5, 7, 7, 5, 7, 7, 5, 7, 7, 5], 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13]
29
[120, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 72, 72, 72, 12, 17, 27, 18, 12, 17, 27, 18, 20, 20, 1, 1, 1, 1, 6]
29
['Share and Growth By Manufacturer/Brands', 'Share And Growth By Manufacturer By Retailer', 'Share And Growth By Brands By Retailer', 'Share And Growth By Manufacturer By Channel', 'Share And Growth By Brands By Channel', 'Share

In [96]:
print(sum(duplication))

544


In [97]:
slideDuplication(index,duplication,section_names,path,new_pre)

In [76]:
prs = Presentation(new_pre)

def calculate_position(end):
    return sum(duplication[i] * (1 if isinstance(index[i], int) else len(index[i])) for i in range(end))

#### Slide5_Share_Growth

In [77]:
for k in modified_brands_P12M_new.keys():
    t=modified_brands_P12M_new[k].copy()
    t= t.rename(columns = { "Value Share P12M":"Value Share", "Share DYA P12M":"Share DYA", "Av Price/KG P12M":"Av Price/KG"})
    modified_brands_P12M_new[k] = t

In [78]:
p = 0
Share_Growth(prs, list(modified_manuf_P12M_new.keys()), modified_manuf_P12M_new, modified_brands_P12M_new, position =calculate_position(p))
p+=1

In [79]:
for k in modified_brands_P12M_new.keys():
    t=modified_brands_P12M_new[k].copy()
    t= t.rename(columns = {"Value Share": "Value Share P12M", "Share DYA":"Share DYA P12M", "Av Price/KG":"Av Price/KG P12M"})
    modified_brands_P12M_new[k] = t[t['Value Share P12M'] != 0]


### Slide6_Share and Growth By Manufacturer 

In [80]:
c = calculate_position(p)
if len(regions_RET)!=0:
    for key in shareGrowthDf_retailers_REG.keys():
        retailers_REG_list = [shareGrowthDf_retailers_REG[key][i:i+3] for i in range(0, len(shareGrowthDf_retailers_REG[key]), 3)]
        for sublist in retailers_REG_list:
            Share_Growth_Separately_area( prs,[key], modified_manuf_P12M_new,position=c,slide_by='Manufacturer | By Retailer', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_retailers_REG.keys():
        retailers_REG_list = [shareGrowthDf_retailers_REG[key][i:i+3] for i in range(0, len(shareGrowthDf_retailers_REG[key]), 3)]
        for sublist in retailers_REG_list:
            Share_Growth_Separately_area( prs,[key], modified_brands_P12M_new,position=c,slide_by='Brands | By Retailer', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1
    p+=1 
if len(channels_RET)!=0:
    c=calculate_position(p) 
    for key in shareGrowthDf_retailers_CHAN.keys():
        retailers_CHAN_list = [shareGrowthDf_retailers_CHAN[key][i:i+3] for i in range(0, len(shareGrowthDf_retailers_CHAN[key]), 3)]
        for sublist in retailers_CHAN_list:
            Share_Growth_Separately_area( prs,[key], modified_manuf_P12M_new,position=c,slide_by='Manufacturer | By Retailer', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_retailers_CHAN.keys():
        retailers_CHAN_list = [shareGrowthDf_retailers_CHAN[key][i:i+3] for i in range(0, len(shareGrowthDf_retailers_CHAN[key]), 3)]
        for sublist in retailers_CHAN_list:
            Share_Growth_Separately_area( prs,[key], modified_brands_P12M_new,position=c,slide_by='Brands | By Retailer', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  
    p+=1
if len(market_RET)!=0:
    c=calculate_position(p) 
    for key in shareGrowthDf_retailers_MAR.keys():
        retailers_MAR_list = [shareGrowthDf_retailers_MAR[key][i:i+3] for i in range(0, len(shareGrowthDf_retailers_MAR[key]), 3)]
        for sublist in retailers_MAR_list:
            Share_Growth_Separately_area( prs,[key], modified_manuf_P12M_new,position=c,slide_by='Manufacturer | By Retailer', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_retailers_MAR.keys():
        retailers_MAR_list = [shareGrowthDf_retailers_MAR[key][i:i+3] for i in range(0, len(shareGrowthDf_retailers_MAR[key]), 3)]
        for sublist in retailers_MAR_list:
            Share_Growth_Separately_area( prs,[key], modified_brands_P12M_new,position=c,slide_by='Brands | By Retailer', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1 
    p+=1
    

Yogurt
Functionals
Every Day Nutrition
Plant Based
Protein
Bifidus
Cholesterol
Essential
Greek
Immunity
Kefir
Kids
Light
Essential Drink
Essential Spoon
Greek Spoon
Kids Drink
Kids Spoon
Light Drink
Light Spoon
Yogurt
Functionals
Every Day Nutrition
Plant Based
Protein
Bifidus
Cholesterol
Essential
Greek
Immunity
Kefir
Kids
Light
Essential Drink
Essential Spoon
Greek Spoon
Kids Drink
Kids Spoon
Light Drink
Light Spoon


In [81]:
c=calculate_position(p)  
if len(regions_CHAN)!=0:
    for key in shareGrowthDf_channels_REG.keys():
        channels_REG_list = [shareGrowthDf_channels_REG[key][i:i+3] for i in range(0, len(shareGrowthDf_channels_REG[key]), 3)]
        for sublist in channels_REG_list:
            Share_Growth_Separately_area( prs,[key], modified_manuf_P12M_new,position=c,slide_by='Manufacturer | By Channel', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_channels_REG.keys():
        channels_REG_list = [shareGrowthDf_channels_REG[key][i:i+3] for i in range(0, len(shareGrowthDf_channels_REG[key]), 3)]
        for sublist in channels_REG_list:
            Share_Growth_Separately_area( prs,[key], modified_brands_P12M_new,position=c,slide_by='Brands | By Channel', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1
    p+=1   
if len(channels_CHAN)!=0:  
    c=calculate_position(p) 
    for key in shareGrowthDf_channels_CHAN.keys():
        channels_CHAN_list = [shareGrowthDf_channels_CHAN[key][i:i+3] for i in range(0, len(shareGrowthDf_channels_CHAN[key]), 3)]
        for sublist in channels_CHAN_list:
            Share_Growth_Separately_area( prs,[key], modified_manuf_P12M_new,position=c,slide_by='Manufacturer | By Channel', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_channels_CHAN.keys():
        channels_CHAN_list = [shareGrowthDf_channels_CHAN[key][i:i+3] for i in range(0,len(shareGrowthDf_channels_CHAN[key]), 3)]
        for sublist in channels_CHAN_list:
            Share_Growth_Separately_area( prs,[key], modified_brands_P12M_new,position=c,slide_by='Brands | By Channel', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  
    p+=1
if len(market_CHAN)!=0:
    c=calculate_position(p) 
    for key in shareGrowthDf_channels_MAR.keys():
        channels_MAR_list = [shareGrowthDf_channels_MAR[key][i:i+3] for i in range(0, len(shareGrowthDf_channels_MAR[key]), 3)]
        for sublist in channels_MAR_list:
            Share_Growth_Separately_area( prs,[key], modified_manuf_P12M_new,position=c,slide_by='Manufacturer | By Channel', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_channels_MAR.keys():
        channels_MAR_list = [shareGrowthDf_channels_MAR[key][i:i+3] for i in range(0, len(shareGrowthDf_channels_MAR[key]), 3)]
        for sublist in channels_MAR_list:
            Share_Growth_Separately_area( prs,[key], modified_brands_P12M_new,position=c,slide_by='Brands | By Channel', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1 
    p+=1

Yogurt
Functionals
Every Day Nutrition
Plant Based
Protein
Bifidus
Cholesterol
Essential
Greek
Immunity
Kefir
Kids
Light
Essential Drink
Essential Spoon
Greek Spoon
Kids Drink
Kids Spoon
Light Drink
Light Spoon
Yogurt
Functionals
Every Day Nutrition
Plant Based
Protein
Bifidus
Cholesterol
Essential
Greek
Immunity
Kefir
Kids
Light
Essential Drink
Essential Spoon
Greek Spoon
Kids Drink
Kids Spoon
Light Drink
Light Spoon


In [82]:
c=calculate_position(p)  
if len(regions_CUST)!=0:
    for key in shareGrowthDf_custom_REG.keys():
        custom_REG_list = [shareGrowthDf_custom_REG[key][i:i+3] for i in range(0, len(shareGrowthDf_custom_REG[key]), 3)]
        for sublist in custom_REG_list:
            Share_Growth_Separately_area( prs,[key], modified_manuf_P12M_new,position=c,slide_by='Manufacturer | By '+ customareas, list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_custom_REG.keys():
        custom_REG_list = [shareGrowthDf_custom_REG[key][i:i+3] for i in range(0, len(shareGrowthDf_custom_REG[key]), 3)]
        for sublist in custom_REG_list:
            Share_Growth_Separately_area( prs,[key], modified_brands_P12M_new,position=c,slide_by='Brands | By '+ customareas, list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1
    p+=1     
if len(channels_CUST)!=0:
    c=calculate_position(p) 
    for key in shareGrowthDf_custom_CHAN.keys():
        custom_CHAN_list = [shareGrowthDf_custom_CHAN[key][i:i+3] for i in range(0, len(shareGrowthDf_custom_CHAN[key]), 3)]
        for sublist in custom_CHAN_list:
            Share_Growth_Separately_area( prs,[key], modified_manuf_P12M_new,position=c,slide_by='Manufacturer | By '+ customareas, list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_custom_CHAN.keys():
        custom_CHAN_list = [shareGrowthDf_custom_CHAN[key][i:i+3] for i in range(0, len(shareGrowthDf_custom_CHAN[key]), 3)]
        for sublist in custom_CHAN_list:
            Share_Growth_Separately_area( prs,[key], modified_brands_P12M_new,position=c,slide_by='Brands | By '+ customareas, list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  
    p+=1
if len(market_CUST)!=0:
    c=calculate_position(p) 
    for key in shareGrowthDf_custom_MAR.keys():
        custom_MAR_list = [shareGrowthDf_custom_MAR[key][i:i+3] for i in range(0, len(shareGrowthDf_custom_MAR[key]), 3)]
        for sublist in custom_MAR_list:
            Share_Growth_Separately_area( prs,[key], modified_manuf_P12M_new,position=c,slide_by='Manufacturer | By '+ customareas, list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_custom_MAR.keys():
        custom_MAR_list = [shareGrowthDf_custom_MAR[key][i:i+3] for i in range(0, len(shareGrowthDf_custom_MAR[key]), 3)]
        for sublist in custom_MAR_list:
            Share_Growth_Separately_area( prs,[key], modified_brands_P12M_new,position=c,slide_by='Brands | By '+ customareas, list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1 
    p+=1

In [83]:
c = calculate_position(p)
if len(sectors)!=0:
    for key in sharGrowthDf_sec.keys():
        sectors_list = [sharGrowthDf_sec[key][i:i+3] for i in range(0, len(sharGrowthDf_sec[key]), 3)]
        print(sectors_list)
        for sublist in sectors_list:
            print(sublist)
            Share_Growth_Separately( prs,[key], modified_manuf_P12M_new,position=c,slide_by='Manufacturer | By Sector', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1

    c=calculate_position(p)
    for key in sharGrowthDf_sec.keys():
        sectors_list = [sharGrowthDf_sec[key][i:i+3] for i in range(0, len(sharGrowthDf_sec[key]), 3)]
        for sublist in sectors_list:
            Share_Growth_Separately( prs,[key], modified_brands_P12M_new,position=c,slide_by='Brands | By Sector', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  
    p+=1
if len(segments)!=0:
    c = calculate_position(p)
    for key in sharGrowthDf_seg.keys():
        segments_list = [sharGrowthDf_seg[key][i:i+3] for i in range(0, len(sharGrowthDf_seg[key]), 3)]
        for sublist in segments_list:
            Share_Growth_Separately( prs,[key], modified_manuf_P12M_new,position=c,slide_by='Manufacturer | By Segment', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    
    c=calculate_position(p)
    for key in sharGrowthDf_seg.keys():
        segments_list = [sharGrowthDf_seg[key][i:i+3] for i in range(0, len(sharGrowthDf_seg[key]), 3)]
        for sublist in segments_list:
            Share_Growth_Separately( prs,[key], modified_brands_P12M_new,position=c,slide_by='Brands | By Segment', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  # Correctly incrementing the variable c
    p+=1

if len(subsegments)!=0:
    c = calculate_position(p)
    for key in sharGrowthDf_subseg.keys():
        subsegments_list = [sharGrowthDf_subseg[key][i:i+3] for i in range(0, len(sharGrowthDf_subseg[key]), 3)]
        for sublist in subsegments_list:
            Share_Growth_Separately( prs,[key], modified_manuf_P12M_new,position=c,slide_by='Manufacturer | By SubSegment', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  # Correctly incrementing the variable c
    p+=1
    
    c=calculate_position(p)
    for key in sharGrowthDf_subseg.keys():
        subsegments_list = [sharGrowthDf_subseg[key][i:i+3] for i in range(0, len(sharGrowthDf_subseg[key]), 3)]
        for sublist in subsegments_list:
            Share_Growth_Separately( prs,[key], modified_brands_P12M_new,position=c,slide_by='Brands | By SubSegment', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  # Correctly incrementing the variable c
    p+=1


if len(subcategories)!=0:
    c = calculate_position(p)
    for key in sharGrowthDf_subcat.keys():
        subcateg_list = [sharGrowthDf_subcat[key][i:i+3] for i in range(0, len(sharGrowthDf_subcat[key]), 3)]
        for sublist in subcateg_list:
            Share_Growth_Separately( prs,[key], modified_manuf_P12M_new,position=c,slide_by='Manufacturer | By SubCategory', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  # Correctly incrementing the variable c
    p+=1
    
    c=calculate_position(p)
    for key in sharGrowthDf_subcat.keys():
        subcateg_list = [sharGrowthDf_subcat[key][i:i+3] for i in range(0, len(sharGrowthDf_subcat[key]), 3)]
        for sublist in subcateg_list:
            Share_Growth_Separately( prs,[key], modified_brands_P12M_new,position=c,slide_by='Brands | By SubCategory', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  # Correctly incrementing the variable c
    p+=1            

[['Protein', 'Functionals', 'Plant Based'], ['Every Day Nutrition']]
['Protein', 'Functionals', 'Plant Based']
NATIONAL
['Every Day Nutrition']
NATIONAL
[['Protein', 'Plant Based', 'Every Day Nutrition'], ['Functionals']]
['Protein', 'Plant Based', 'Every Day Nutrition']
Ahorramas
['Functionals']
Ahorramas
[['Functionals', 'Plant Based', 'Protein'], ['Every Day Nutrition']]
['Functionals', 'Plant Based', 'Protein']
Carrefour
['Every Day Nutrition']
Carrefour
[['Functionals', 'Plant Based', 'Every Day Nutrition'], ['Protein']]
['Functionals', 'Plant Based', 'Every Day Nutrition']
Discounters
['Protein']
Discounters
[['Plant Based', 'Protein', 'Functionals'], ['Every Day Nutrition']]
['Plant Based', 'Protein', 'Functionals']
Hyper
['Every Day Nutrition']
Hyper
[['Protein', 'Plant Based', 'Functionals'], ['Every Day Nutrition']]
['Protein', 'Plant Based', 'Functionals']
Super
['Every Day Nutrition']
Super
NATIONAL
NATIONAL
Ahorramas
Ahorramas
Carrefour
Carrefour
Discounters
Discounters
Hy

## Slide 12 : Category Evolution Analysis 

In [84]:
manuf_brand_list = client_manuf + client_brands

In [85]:
if sectors:
    categoryEvolution(prs,cea_sectors_clients_new,cea_sectors_P12M_new,cea_dya_sectors_new,cea_sectors_clients_total_new,manuf_brand_list,col="Sector",position = calculate_position(p))
    p+=1

In [86]:
if len(segments)!=0:
    posadd=calculate_position(p)
    categoryEvolution(prs,cea_segment_clients_new,cea_segment_P12M_new,cea_dya_segment_new,cea_segment_clients_total_new,manuf_brand_list,col='Segment',position = posadd)
    p+=1    

In [87]:
if len(subsegments)!=0:
    posadd=calculate_position(p)
    categoryEvolution(prs,cea_subsegment_clients_new,cea_subsegment_P12M_new,cea_dya_subsegment_new,cea_subsegment_clients_total_new,manuf_brand_list,col='SubSegment',position = posadd)
    p+=1


In [88]:
if len(subcategories)!=0:
    posadd=calculate_position(p)
    categoryEvolution(prs,cea_subcategory_clients_new,cea_subcategory_P12M_new,cea_dya_subcategory_new,cea_subcategory_clients_total_new,manuf_brand_list,col='SubCategory',position = posadd)   
    p+=1  

## Slide 13 : Share And Growth

In [89]:
catSecSeg = (
    categories + sectors + segments + subsegments + subcategories
    if any([categories, sectors, segments, subsegments, subcategories])
    else []
)

###### Retailer

In [90]:
if not any([
    globals().get("modified_retailer_Region", False),
    globals().get("modified_retailer_Channel", False),
    globals().get("modified_retailer_Market", False)
]):
    # Skip the block
    pass
else:
    for key,val in retailerField.items():
        if val[0]:
 
            if categories:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in retailerCatDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in categories}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
            if sectors:  
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in retailerSecDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in sectors}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
 
            if segments:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in retailerSegDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in segments}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
 
            if subsegments:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in retailerSubSegDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in subsegments}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
           
            if subcategories:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in retailerSubCatDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in subcategories}
 
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1

###### Channel

In [91]:
if not any([
    globals().get("modified_channels_Region", False),
    globals().get("modified_channels_Channel", False),
    globals().get("modified_channels_Market", False)
]):
    # Skip the block
    pass
else:
    for key,val in channelField.items():
        if val[0]:
            if categories:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in channelCatDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in categories}
               
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
           
 
            if sectors:  
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in channelSecDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in sectors}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
            if segments:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in channelSegDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in segments}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
 
            if subsegments:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in channelSubSegDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in subsegments}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
            if subcategories:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in channelSubCatDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in subcategories}
 
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1

###### Custom

In [92]:
if not any([
    globals().get("modified_cust_Region", False),
    globals().get("modified_cust_Channel", False),
    globals().get("modified_cust_Market", False)
]):
    # Skip the block
    pass
else:
    for key,val in custField.items():
        if val[0]:
 
            if categories:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1] if key in custCatDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0] if key in categories}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
            if sectors:  
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1] if key in custSecDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0] if key in sectors}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
 
            if segments:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1] if key in custSegDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0] if key in segments}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
            if subsegments:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1] if key in custSubSegDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0] if key in subsegments}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
           
            if subcategories:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1] if key in custSubCatDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0] if key in subcategories}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1

#### Momentum Analysis

In [93]:

if retailers:
    Momentum_Analysis(prs, list(modified_momentum_retailers.keys()), modified_momentum_retailers, position = calculate_position(p)) 
    p+=1

if channels:
    Momentum_Analysis(prs, list(modified_momentum_channels.keys()), modified_momentum_channels, position = calculate_position(p)) 
    p+=1

if cust:
    Momentum_Analysis(prs, list(modified_momentum_cust.keys()), modified_momentum_cust, position = calculate_position(p)) 
    p+=1


if segments:
    if retailers:
        Momentum_Analysis(prs, list(modified_momentum_retailers_seg.keys()), modified_momentum_retailers_seg, position = calculate_position(p)) 
        p+=1

    if channels:
        Momentum_Analysis(prs, list(modified_momentum_channels_seg.keys()), modified_momentum_channels_seg, position = calculate_position(p)) 
        p+=1

    if cust:
        Momentum_Analysis(prs, list(modified_momentum_cust_seg.keys()), modified_momentum_cust_seg, position = calculate_position(p)) 
        p+=1

if subsegments:
    if retailers:
        Momentum_Analysis(prs, list(modified_momentum_retailers_subseg.keys()), modified_momentum_retailers_subseg, position = calculate_position(p)) 
        p+=1

    if channels:
        Momentum_Analysis(prs, list(modified_momentum_channels_subseg.keys()), modified_momentum_channels_subseg, position = calculate_position(p)) 
        p+=1

    if cust:
        Momentum_Analysis(prs, list(modified_momentum_cust_subseg.keys()), modified_momentum_cust_subseg, position = calculate_position(p)) 
        p+=1

if subcategories:
    if retailers:
        Momentum_Analysis(prs, list(modified_momentum_retailers_subcatg.keys()), modified_momentum_retailers_subcatg, position = calculate_position(p)) 
        p+=1

    if channels:
        Momentum_Analysis(prs, list(modified_momentum_channels_subcatg.keys()), modified_momentum_channels_subcatg, position = calculate_position(p)) 
        p+=1

    if cust:
        Momentum_Analysis(prs, list(modified_momentum_cust_subcatg.keys()), modified_momentum_cust_subcatg, position = calculate_position(p)) 
        p+=1


Momentum_Analysis(prs, list(modified_momentum_dfs.keys()), modified_momentum_dfs, position = calculate_position(p))

In [94]:
outputPath = os.getcwd()+f"\\Landscape Pre-Output ShareGrowth_{datetime.today().strftime("%d-%m")}.pptx"
prs.save(outputPath)

In [95]:
output_path_final = os.getcwd()+f"\\Final Output\\Landscape ShareGrowth_{datetime.today().strftime("%d-%m")}.pptx"
open_chart_data_in_excel(outputPath,output_path_final)

Slide 673: Opened Excel workbook: Book1
Slide 674: Opened Excel workbook: Book1
Slide 675: Opened Excel workbook: Book1
Slide 676: Opened Excel workbook: Book1
Slide 677: Opened Excel workbook: Book1
Slide 678: Opened Excel workbook: Book1
Slide 679: Opened Excel workbook: Book1
Slide 680: Opened Excel workbook: Book1
Slide 681: Opened Excel workbook: Book1
Slide 682: Opened Excel workbook: Book1
Slide 683: Opened Excel workbook: Book1
Slide 684: Opened Excel workbook: Book1
Slide 685: Opened Excel workbook: Book1
Slide 686: Opened Excel workbook: Book1
Slide 687: Opened Excel workbook: Book1
Slide 688: Opened Excel workbook: Book1
Slide 689: Opened Excel workbook: Book1
Slide 690: Opened Excel workbook: Book1
Slide 691: Opened Excel workbook: Book1
Slide 692: Opened Excel workbook: Book1
Slide 693: Opened Excel workbook: Book1
Slide 694: Opened Excel workbook: Book1
Slide 695: Opened Excel workbook: Book1
Slide 696: Opened Excel workbook: Book1
Slide 697: Opened Excel workbook: Book1


# RPVM

In [49]:
variables = [
    "revenue_PVM_manuf",
    "revenue_PVM_brand"
]

for var in variables:
    if var in globals():
        globals()[var] = add_totals_after_grouping(globals()[var])


In [50]:
modified_revenue_PVM_Manuf = {}

for key in revenue_PVM_manuf.keys():
    df = revenue_PVM_manuf[key].copy()
    df['Value Share'] = df['Value Share'].astype(float) 
    df['Revenue By PVM'] = df['Revenue By PVM'].astype(float) / 1000
    relevant_client_brands = [brand for brand in client_manuf if brand in df[f"{ManufOrTopC}"].values]
    if relevant_client_brands:
        filtered_df = df[
            (df[f"{ManufOrTopC}"].str.contains("Total")) & 
            (df[f"{ManufOrTopC}"] != "Grand Total") &
            (df[f"{ManufOrTopC}"] != "All Others Total") &  
            (~df[f"{ManufOrTopC}"].str.replace(" Total", "").isin(client_manuf))
        ].nlargest(1, "Value Share")
        if not filtered_df.empty:
            top_competitor = filtered_df[f"{ManufOrTopC}"].str.replace(" Total", "").iloc[0]
        else:
            top_competitor = None  
        if top_competitor:
            df[f"{ManufOrTopC}"] = df[f"{ManufOrTopC}"].str.strip()
            for brand in relevant_client_brands:
                filtered_df = df[(df[f"{ManufOrTopC}"] == brand) | (df[f"{ManufOrTopC}"] == top_competitor)]
                new_key = f"{key} | {brand}"
                if not filtered_df.empty:
                     modified_revenue_PVM_Manuf[new_key] = filtered_df

In [51]:
modified_revenue_PVM_Brand = {}

for key in revenue_PVM_brand.keys():
    df = revenue_PVM_brand[key].copy()
    df['Value Share'] = df['Value Share'].astype(float)
    df['Revenue By PVM'] = df['Revenue By PVM'].astype(float) / 1000
    relevant_client_brands = [brand for brand in client_brands if brand in df[f'{BrandOrTopB}'].values]
    if relevant_client_brands:
        filtered_df = df[
            (df[f"{BrandOrTopB}"].str.contains(" Total")) & 
            (df[f"{BrandOrTopB}"] != "Grand Total") &
            (df[f"{BrandOrTopB}"] != "All Others Total") &  
            (~df[f"{BrandOrTopB}"].str.replace(" Total", "").isin(client_brands))
        ].nlargest(1, "Value Share")
        if not filtered_df.empty:
            top_competitor = filtered_df[f"{BrandOrTopB}"].str.replace(" Total", "").iloc[0]
        else:
            top_competitor = None  
        if top_competitor:
            df[f"{BrandOrTopB}"] = df[f"{BrandOrTopB}"].str.strip()
            for brand in relevant_client_brands:
                filtered_df = df[(df[f'{BrandOrTopB}'] == brand) | (df[f'{BrandOrTopB}'] == top_competitor)]
                new_key = f"{key} | {brand}"
                if not filtered_df.empty:
                    modified_revenue_PVM_Brand[new_key] = filtered_df

In [52]:
index = [*[14]*(1)]
index=[i for i in index if i !=[]]

duplication_1 = [len(modified_revenue_PVM_Manuf.keys()) +len(modified_revenue_PVM_Brand.keys())]
# duplication_2=[len(modified_revenue_PVM_Brand.keys())]

duplication = duplication_1


duplication = [item for item in duplication if item !=0]

section_names_slide1 = ["Revenue PVM By Manufacturer & Brands"]
# section_names_slide2 = ["Revenue PVM By Brands"]

section_names = [ *section_names_slide1]


In [53]:
path = os.getcwd() + '\Landscape base.pptx'
new_pre = os.getcwd() + '\Landscape Duplicate RPVM.pptx'

In [54]:
r=modified_revenue_PVM_Brand.keys()

In [55]:
len(modified_revenue_PVM_Brand.keys())

281

In [56]:
print(len(index))
print(index)
print(len(duplication))
print(duplication)
print(len(section_names))
print(section_names)
print(sum(duplication))

1
[14]
1
[400]
1
['Revenue PVM By Manufacturer & Brands']
400


In [57]:
slideDuplication(index,duplication,section_names,path,new_pre)

In [58]:
def calculate_position(end):
    return sum(duplication[i] * (1 if isinstance(index[i], int) else len(index[i])) for i in range(end))


ppt_app = win32com.client.Dispatch("PowerPoint.Application")

prs = ppt_app.Presentations.open(new_pre)


In [59]:
combined_list_duplicates = list(modified_revenue_PVM_Manuf.keys())+ list(modified_revenue_PVM_Brand.keys())
combined_modified_RPVM = {**modified_revenue_PVM_Manuf, **modified_revenue_PVM_Brand}

RPVM(prs, combined_list_duplicates, combined_modified_RPVM, position=calculate_position(0))


Error updating chart data on slide 1: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147467259), None)
Error on slide 1, retry 1: (-2147023174, 'The RPC server is unavailable.', None, None)
Error updating chart data on slide 1: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147467259), None)
Error on slide 1, retry 2: (-2147023174, 'The RPC server is unavailable.', None, None)
Error updating chart data on slide 1: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147467259), None)
Error on slide 1, retry 3: (-2147023174, 'The RPC server is unavailable.', None, None)
Error updating chart data on slide 1: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147467259), None)
Error on slide 1, retry 4: (-2147023174, 'The RPC server is unavailable.', None, None)
Error updating chart data on slide 1: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147467259), None)
Error on slide 1, retry 5: (-2147023174, 'The RPC se

KeyboardInterrupt: 

In [ ]:

output_path = os.path.join(os.getcwd(), 'Landscape RPVM Trying.pptx')
prs.SaveAs(output_path)


AttributeError: <unknown>.SaveAs

# Slides WITHOUT forcing CLIENT BRAND

In [ ]:
Sections_WO_CB=True

In [ ]:
if not Sections_WO_CB:
    print("Stopping the script as 'Sections_WO_CB' is set to False.")
    sys.exit()  # Terminate the script here


In [ ]:
manuf_dfs_new=process_data(manuf_dfs,"Year")
manuf_P12M_dfs_new=process_data(manuf_12_dfs,"Time Period")
brands_dfs_new=process_data(brand_dfs,"Year")

NameError: name 'process_data' is not defined

In [ ]:
modified_brands_share_noCB = {}
modified_brands_totals_noCB = {}
modified_brands_P12M_noCB = {}


for s in brands_dfs_new.keys():
    t = brands_dfs_new[s].copy()
    # t=DetectHeader(t).fillna(0)
    t.columns = t.columns.str.strip()
   
    t = t.applymap(lambda x: float(x) if pd.to_numeric(x, errors='coerce') == x else x)
    t = t.applymap(lambda x: 0 if pd.isna(x) else x)
    mod = t[(~t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    mod = mod.sort_values([col for col in mod.columns if 'Value Share' in col], ascending=False)
    tot = t[(t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    if not mod.empty:
        modified_brands_share_noCB[s] = mod
    if not tot.empty:
        modified_brands_totals_noCB[s] = tot


df_toDrop =[key for key in modified_brands_share_noCB.keys() if (modified_brands_share_noCB[key].shape[1]>2) and (modified_brands_share_noCB[key].shape[1]<4)]
for key in df_toDrop:
    del modified_brands_share_noCB[key]

    
emptyDf=[key for key in modified_brands_share_noCB.keys() if modified_brands_share_noCB[key].shape[1]<4]
for dic in [modified_brands_share_noCB]:
    if emptyDf:
        columnsToAdd=list(set(dic.keys())-set(emptyDf))[0]
        for df in emptyDf:
            print()
            dic[df]=dic[df].drop(columns=[''])#list(set(dic[df].columns)-set(['']))
            emptyCol=list(set(dic[columnsToAdd].columns)-set(dic[df].columns)-set(['']))
            dic[df][emptyCol]=0.0
p = 10 if len(client_brands) < 10 else len(client_brands) + 1

modified_brands_share_noCB = dfSort(modified_brands_share_noCB, [],f'{BrandOrTopB}',p, 'Total Value Share')
for key in modified_brands_share_noCB.keys():
    modified_brands_share_noCB[key] = modified_brands_share_noCB[key][modified_brands_share_noCB[key]['Total Value Share'] != 0]
addOther(modified_brands_share_noCB,col=f'{BrandOrTopB}',TorB="Top")


for s in brand_12_dfs.keys():
    t = brand_12_dfs[s].copy()
    t.columns = t.columns.str.strip()
    t = t.applymap(lambda x: float(x) if pd.to_numeric(x, errors='coerce') == x else x)
    t = t.applymap(lambda x: 0 if pd.isna(x) else x)
    mod = t[(~t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    mod = mod.sort_values([col for col in mod.columns if 'Value Share' in col], ascending=False)
    tot = t[(t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    modified_brands_P12M_noCB[s] = mod



df_toDrop =[key for key in modified_brands_P12M_noCB.keys() if (modified_brands_P12M_noCB[key].shape[1]>2) and (modified_brands_P12M_noCB[key].shape[1]<4)]
for key in df_toDrop:
    del modified_brands_P12M_noCB[key]

emptyDf=[key for key in modified_brands_P12M_noCB.keys() if modified_brands_P12M_noCB[key].shape[1]<4]
for dic in [modified_brands_P12M_noCB]:
    if emptyDf:
        columnsToAdd=list(set(dic.keys())-set(emptyDf))[0]
        for df in emptyDf:
            dic[df]=dic[df].drop(columns=[''])#list(set(dic[df].columns)-set(['']))
            emptyCol=list(set(dic[columnsToAdd].columns)-set(dic[df].columns)-set(['']))
            dic[df][emptyCol]=0.0

modified_brands_P12M_noCB = dfSort(modified_brands_P12M_noCB, [],f'{BrandOrTopB}',10, 'Value Share')


In [ ]:
index = [
    *[0]*(1),
           *[1]*(1),
                4,
                sectorIndex if sectors else [],
                segmentIndex if segments else [],
                subsegmentIndex if subsegments else [],
                subcategoryIndex if subcategories else []
                ]
index=[i for i in index if i !=[]]

duplication_1 = [len(modified_brands_share_noCB.keys())]
duplication_2 = [len(modified_brands_share_noCB.keys())]

duplication_5 = [len(modified_manuf_P12M_new.keys())] 
duplication_6 = [1 if sectors else 0, 1 if segments else 0,  1 if subsegments else 0,  1 if subcategories else 0]

duplication = duplication_1 + duplication_2  + duplication_5 + duplication_6


duplication = [item for item in duplication if item !=0]

section_names_slide1 = ["Market Trends by Brands"]
section_names_slide2 = ["Market Concentration By Brands"]
section_names_slide5 = ['Share and Growth By Manufacturer/Brands']
section_names_slide6 = ([ 'Share And Growth By Brands By Sector'] if len(sectors)>0 else [] ) + (['Share And Growth By Brands By Segment'] if len(segments)>0 else [] ) + (['Share And Growth By Brands By SubSegment'] if len(subsegments)>0 else [] )  + (['Share And Growth By Brands By SubCategory'] if len(subcategories)>0 else [] )

section_names = [ *section_names_slide1,*section_names_slide2,
                  *section_names_slide5, *section_names_slide6  
                 ]


In [ ]:
path = os.getcwd() + '\Landscape base.pptx'
new_pre = os.getcwd() + '\Landscape Duplicate Without CB.pptx'

In [ ]:
print(len(index))
# print(index)
print(len(duplication))
print(duplication)
print(len(section_names))
# print(section_names)
print(sum(duplication))

In [ ]:
# slideDuplication(index,duplication,section_names,path,new_pre)


In [ ]:
def calculate_position(end):
    return sum(duplication[i] * (1 if isinstance(index[i], int) else len(index[i])) for i in range(end))

In [ ]:
prs = Presentation(new_pre)


In [ ]:
p = 0
Market_Trends(prs, list(modified_brands_share_noCB.keys()), modified_brands_share_noCB, modified_brands_totals_noCB, client_brands ,position =calculate_position(p), slide_by = f'{BrandOrTopB}', years=years)

p+=1

In [ ]:
Market_Concentration(prs,list(modified_brands_share_noCB.keys()), modified_brands_share_noCB, modified_brands_totals_noCB, client_brands ,position =  calculate_position(p), slide_by = f'{BrandOrTopB}',years=years)

p+=1

In [ ]:
for k in modified_brands_P12M_noCB.keys():
    modified_brands_P12M_noCB[k] = modified_brands_P12M_noCB[k].rename(columns = { "Value Share P12M":"Value Share", "Share DYA P12M":"Share DYA", "Av Price/KG P12M":"Av Price/KG"})
    modified_brands_P12M_noCB[k] = modified_brands_P12M_noCB[k].sort_values('Value Share', ascending = False)
    modified_brands_P12M_noCB[k] = modified_brands_P12M_noCB[k][modified_brands_P12M_noCB[k]['Value Share']!=0]    
    modified_brands_P12M_noCB[k]['Sorting'] = np.where(modified_brands_P12M_noCB[k][f'{BrandOrTopB}'] == 'Others', 1, 0)
    modified_brands_P12M_noCB[k] = modified_brands_P12M_noCB[k].sort_values(['Sorting',"Value Share"], ascending = [True, False]).drop(columns = 'Sorting')


In [ ]:
Share_Growth(prs, list(modified_manuf_P12M_new.keys()), modified_manuf_P12M_new, modified_brands_P12M_noCB, position =calculate_position(p))

p+=1

In [ ]:
for k in modified_brands_P12M_noCB.keys():
    t=modified_brands_P12M_noCB[k].copy()
    t= t.rename(columns = {"Value Share": "Value Share P12M", "Share DYA":"Share DYA P12M", "Av Price/KG":"Av Price/KG P12M"})
    modified_brands_P12M_noCB[k] = t[t['Value Share P12M'] != 0]

In [ ]:
c = calculate_position(p)
if len(sectors)!=0:

    c=calculate_position(p)
    for key in sharGrowthDf_sec.keys():
        sectors_list = [sharGrowthDf_sec[key][i:i+3] for i in range(0, len(sharGrowthDf_sec[key]), 3)]
        for sublist in sectors_list:
            Share_Growth_Separately( prs,[key], modified_brands_P12M_noCB,position=c,slide_by='Brands | By Sector', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  
    p+=1
if len(segments)!=0:
    
    c=calculate_position(p)
    for key in sharGrowthDf_seg.keys():
        segments_list = [sharGrowthDf_seg[key][i:i+3] for i in range(0, len(sharGrowthDf_seg[key]), 3)]
        for sublist in segments_list:
            Share_Growth_Separately( prs,[key], modified_brands_P12M_noCB,position=c,slide_by='Brands | By Segment', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  # Correctly incrementing the variable c
    p+=1

if len(subsegments)!=0:
    
    c=calculate_position(p)
    for key in sharGrowthDf_subseg.keys():
        subsegments_list = [sharGrowthDf_subseg[key][i:i+3] for i in range(0, len(sharGrowthDf_subseg[key]), 3)]
        for sublist in subsegments_list:
            Share_Growth_Separately( prs,[key], modified_brands_P12M_noCB,position=c,slide_by='Brands | By SubSegment', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  # Correctly incrementing the variable c
    p+=1


if len(subcategories)!=0:
    
    c=calculate_position(p)
    for key in sharGrowthDf_subcat.keys():
        subcateg_list = [sharGrowthDf_subcat[key][i:i+3] for i in range(0, len(sharGrowthDf_subcat[key]), 3)]
        for sublist in subcateg_list:
            Share_Growth_Separately( prs,[key], modified_brands_P12M_noCB,position=c,slide_by='Brands | By SubCategory', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  # Correctly incrementing the variable c
    p+=1        

In [ ]:
outputPath = os.getcwd()+f"\\Final Output\\Landscape Output WO CB_{datetime.today().strftime("%d-%m")}.pptx"
prs.save(outputPath)

In [ ]:
end_time = time.time()

elapsed_time = end_time - start_time
print(f"Script started at: {time.ctime(start_time)}")
print(f"Script ended at: {time.ctime(end_time)}")
print(f"Elapsed time: {elapsed_time:.2f} seconds")